# Machine Learning in Computational Biology
## Course Project
## 4 class problem classification - Pipeline and Results Notebook
#### Papadopoulou Marianna, ID:7115152200032
#### Vossos Charalampos, ID:7115152200037
#### Fillipidou Thalassini-Marina, ID:7115152200022

This notebook presents all the pipeline for the 4class classification problem 
of cells based on their gene profile. It contains all the optimization steps, hyperparameter tunning and the feature selection techniques that were tested
In order to replicate the findings outlined in this academic document, please follow the steps below:

1.	Open the notebook provided using Google Colab 
2.	Access the data section of the environment by clicking on the folder icon located on the left side. 
Then, upload the:
  - final_data.csv file <br>
  or the files
  - E-MTAB-6108.aggregated_filtered_normalised_counts.mtx_cols
  - E-MTAB-6108.aggregated_filtered_normalised_counts.mtx_rows
  - E-MTAB-6108.aggregated_filtered_normalised_counts.mtx<br>
  AND 
  - ground_truth.xlsx

 These files are provided within the exercise's zip file. To upload them, simply drag and drop the files into the designated area.

3. Run the cells in the notebook in the order they appear, making sure to follow the instructions provided in each cell.
	 
When executing the notebook in Jupyter, it is essential to ensure that the necessary packages have been installed. Additionally, it is important to specify the path to the dataset (final_data.csv or other three files) as an input parameter for the *pd.read_csv()* functions.

# Libraries 

In [ ]:
# If opened in Colab
#!pip install mrmr_selection
#!pip install optuna

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, matthews_corrcoef, balanced_accuracy_score, f1_score, fbeta_score, recall_score, precision_score, average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import optuna
from optuna.samplers import RandomSampler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import random
from mrmr import mrmr_classif

# Load dataset, Create final df, Dataset exploration (if we do not have the final_df.csv)

In [4]:
#Select the direcotry where the mtx files exist
with open("content/E-MTAB-6108.aggregated_filtered_normalised_counts.mtx") as f:
    lines = f.readlines()

In [5]:
#loading the data of the mtx into a dataframe
normalized_counts_mtx = {"gene": [], "cell": [], "counts": []}
for line in lines[2:]:
    normalized_counts_mtx["gene"].append(line.split()[0])
    normalized_counts_mtx["cell"].append(line.split()[1])
    normalized_counts_mtx["counts"].append(line.split()[2])
normalized_counts_mtx = pd.DataFrame(normalized_counts_mtx).astype(float)

In [6]:
cols= int(normalized_counts_mtx["gene"].max())
rows = int(normalized_counts_mtx["cell"].max())
matrix = np.zeros((rows, cols))

for i in range(normalized_counts_mtx.shape[0]):
    gene = int(normalized_counts_mtx.loc[i, "gene"]) - 1
    cell = int(normalized_counts_mtx.loc[i, "cell"]) - 1
    counts = normalized_counts_mtx.loc[i, "counts"]
    matrix[cell, gene] = counts

In [7]:
df = pd.DataFrame(matrix)

In [8]:
#adding the gene names
genes = []
with open("content/E-MTAB-6108.aggregated_filtered_normalised_counts.mtx_rows") as f:
    lines = f.readlines()
for line in lines:
    genes.append(line.split()[0])
df.columns = genes

In [9]:
#adding the cell identifiers as the first row of the dataset
with open("content/E-MTAB-6108.aggregated_filtered_normalised_counts.mtx_cols") as f:
    lines = f.readlines()
df.insert(loc=0, column='cell', value=lines)
#df["cell"] = lines
df["cell"] = df["cell"].str.rstrip("\n")

## Adding ground truth labels from clustering results

In [10]:
#Adding the cluster labels at the final columns of our dataframe
final_data = pd.DataFrame({})
ground_truth = pd.read_excel("/home/marina/Downloads/ml project/E-MTAB-6108-normalised-files/ground_truth.xlsx")
temp_df = pd.DataFrame({})
for number, column in enumerate(ground_truth.columns[2:]):
    df_clusters = pd.DataFrame({})
    temp_df = pd.DataFrame(df[df["cell"] == column])
    for i in range(ground_truth.shape[0]):
        number_of_clusters =  ground_truth.loc[i, "K"]
        temp_df["cluster_{}".format(number_of_clusters)] = ground_truth.loc[i, column] # cluster_i : the # of clusters, each row has teh cluster that belongs for a certain split given by i
    final_data = pd.concat([final_data, temp_df])

In [11]:
final_data

,cell,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000289716,cluster_2,cluster_4,cluster_7,cluster_8,cluster_11,cluster_18,cluster_23,cluster_31,cluster_37
0,ERR2538859-AAACCTGAGACCACGA,122.99367,0.0,0.000000,0.0,0.00000,0.0,0.000000,30.748417,0.000000,...,0.0,1,3,3,3,2,2,19,17,12
1,ERR2538859-AAACCTGTCTGATACG,215.19258,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,2,2,6,7,7,3,6,6,1
2,ERR2538859-AAACGGGAGTGTTGAA,0.00000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,1,1,1,1,1,14,14,26,26
3,ERR2538859-AAAGATGTCCGAACGC,0.00000,0.0,65.466450,0.0,0.00000,0.0,32.733227,0.000000,65.466450,...,0.0,1,4,5,5,10,18,22,28,33
4,ERR2538859-AAAGTAGGTTAGTGGG,168.26047,0.0,0.000000,0.0,0.00000,0.0,56.086823,0.000000,56.086823,...,0.0,1,4,4,6,5,7,20,11,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,ERR2538860-TTTGGTTTCGTTACGA,0.00000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,1,2,5,4,3,9,3,5,5
1423,ERR2538860-TTTGTCAAGCCCAACC,0.00000,0.0,172.205950,0.0,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,2,2,6,7,7,3,6,25,1
1424,ERR2538860-TTTGTCACATTGGGCC,220.29666,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,1,1,2,2,4,13,13,12,8
1425,ERR2538860-TTTGTCAGTTGCGTTA,0.00000,0.0,55.496975,0.0,0.00000,0.0,0.000000,277.484860,0.000000,...,0.0,1,1,2,2,6,8,7,13,11


# Load Dataset (final_df.csv, ready for use in classification, already filtered with 1500 most variant genes)

If we have already saved the csv file we can begin from this step by directly loading it

In [12]:
#final_data = pd.read_csv('content/dataset.csv')

In [13]:
#final_data.drop(columns=final_data.columns[0], axis=1, inplace=True)

In [14]:
#final_data = pd.read_csv('final_df.csv')

In [15]:
#final_data

# Get the 1500 genes with most variance

In [19]:
# Calculate the variance across cells for each gene
variances = final_data.iloc[:, 1:-9].var(axis=0)

# Sort the genes based on variance in descending order and select the top 1,500 genes
top_genes = variances.sort_values(ascending=False).index[:1500]

# Get the last columns from the original DataFrame
last_columns = final_data.iloc[:, -9:]

# Filter the DataFrame to keep only the top 1,500 genes and the last columns
df_filtered = pd.concat([final_data[['cell'] + list(top_genes)], last_columns], axis=1)

# Optionally, you can reset the index of the filtered DataFrame
df_filtered.reset_index(drop=True, inplace=True)

In [21]:
df_filtered.iloc[:,1:-9]

,ENSG00000205542,ENSG00000198804,ENSG00000167996,ENSG00000198712,ENSG00000156508,ENSG00000087086,ENSG00000075624,ENSG00000229117,ENSG00000026025,ENSG00000140988,...,ENSG00000241553,ENSG00000106399,ENSG00000124570,ENSG00000115241,ENSG00000136750,ENSG00000087191,ENSG00000177570,ENSG00000159593,ENSG00000075292,ENSG00000185565
0,32962.305,9065.20200,8240.5760,5350.2246,4735.2563,5042.7400,4950.4950,9255.0700,3659.0615,5749.6270,...,30.748417,215.238920,61.496834,92.245255,0.000000,122.993670,61.496834,122.993670,61.496834,0.00000
1,34861.200,0.00000,5579.0670,0.0000,16139.4430,4519.0444,3873.4666,12696.3620,3443.0813,7962.1255,...,215.192580,215.192580,0.000000,215.192580,0.000000,0.000000,0.000000,215.192580,0.000000,0.00000
2,17791.770,9193.14500,4845.0840,3431.0170,13334.9840,4739.7560,1061.1393,14643.7230,2193.0212,12343.2730,...,35.371310,0.000000,35.371310,35.371310,0.000000,35.371310,106.783010,106.113940,70.742620,70.74262
3,26513.914,10016.36700,7070.3770,5564.6484,10540.0990,4058.9202,3404.2556,7725.0415,8837.9720,7200.7144,...,0.000000,0.000000,65.466450,130.932900,0.000000,196.399350,65.466450,0.000000,130.932900,65.46645
4,30791.666,18116.04300,7291.2870,7347.3735,5664.7690,5384.3350,2636.0806,8469.1100,3140.8620,4935.0034,...,224.347290,0.000000,56.086823,56.086823,0.000000,56.086823,70.470960,56.086823,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,20833.332,11986.30100,5707.7627,6849.3150,6563.9270,5707.7627,3139.2693,14554.7940,1141.5525,12557.0770,...,0.000000,0.000000,0.000000,285.388120,570.776250,0.000000,0.000000,0.000000,285.388120,0.00000
1423,73876.350,172.20595,13776.4760,0.0000,5510.5903,9126.9150,18598.2420,10160.1510,7577.0615,7232.6500,...,0.000000,0.000000,0.000000,172.205950,0.000000,172.205950,0.000000,0.000000,0.000000,0.00000
1424,22690.557,14539.58000,3524.7466,4038.7722,10060.2140,2570.1277,3671.6110,8003.4385,6094.8745,8224.4080,...,0.000000,0.000000,0.000000,293.728880,0.000000,220.296660,118.785430,73.432220,0.000000,0.00000
1425,54775.516,5771.68550,11874.2080,2830.3457,6992.6187,11654.3640,8047.0615,8713.0250,6160.1640,7714.0796,...,55.496975,55.496975,0.000000,277.484860,55.496975,55.496975,0.000000,0.000000,55.496975,0.00000


In [22]:
#df_filtered.to_csv('final_df.csv',index=False) # save our filtered final file for further use

# Split dataset to features and labels

In [23]:
# Separate features and target variable
X = df_filtered.iloc[:,1:-9]
y_labels = df_filtered.iloc[:,-9:]

In [24]:
#X_features

In [25]:
#y_labels

# 4 class problem

# Split dataset to two sets(training and test)

In [26]:
#get the labels of the 4 cluster results
y = y_labels['cluster_4']


In [27]:
# Standardize the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [28]:
#y = pd.DataFrame(y)

# Convert y to a 1-dimensional array 0,1,2,3
y = np.ravel(y)-1

In [29]:
# Split the dataset into training/validation set and test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X_std, y, test_size=0.2, random_state=42)

# ANOVA for feature selection

## SVC



#### Cross val for hyperparameter tunning and feature selection

In [30]:
# Define the number of folds for cross-validation
n_folds = 5

In [31]:
# Define the objective function for Optuna hyperparameter tuning
def objective_SVC(trial):
  global best_features
  # Define the hyperparameters to be optimized
  C = trial.suggest_float('C', 0.1, 10)
  gamma = trial.suggest_float('gamma', 0.01, 1)
  kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
  degree = trial.suggest_int('degree', 2, 10)
  # Instantiate the classifier with the current hyperparameters
  classifier = SVC(C=C, gamma=gamma,kernel=kernel,degree=degree)

  # Perform cross-validation with feature selection
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []
  feature_selector = SelectKBest(score_func=f_classif, k=300)  # Adjust the number of features as desired
  best_features = None

  for train_index, val_index in kf.split(X_train_val, y_train_val):
      X_train, X_val = X_train_val[train_index], X_train_val[val_index]
      y_train, y_val = y_train_val[train_index], y_train_val[val_index]

      # Perform feature selection on the training set
      X_train_selected = feature_selector.fit_transform(X_train, y_train)
      selected_features = feature_selector.get_support()

      # Apply the same feature selection on the validation set
      X_val_selected = X_val[:, selected_features]

      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train_selected, y_train)
      y_pred = classifier.predict(X_val_selected)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)

  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  if trial.should_prune()==False:
    best_features = selected_features.copy()

  return avg_mcc

In [32]:
# Define the Optuna study
study = optuna.create_study(direction='maximize',sampler=RandomSampler(seed=42))
study.optimize(objective_SVC, n_trials=100)

[I 2023-07-02 23:01:34,247] A new study created in memory with name: no-name-67116f93-b45d-45dc-ba19-5f058f4e45e1
[I 2023-07-02 23:01:34,575] Trial 0 finished with value: 0.8069975928010802 and parameters: {'C': 3.807947176588889, 'gamma': 0.951207163345817, 'kernel': 'linear', 'degree': 2}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:01:35,714] Trial 1 finished with value: 0.0 and parameters: {'C': 8.675143843171858, 'gamma': 0.6051038616257767, 'kernel': 'rbf', 'degree': 3}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:01:36,624] Trial 2 finished with value: 0.20571603821524898 and parameters: {'C': 1.9000671753502962, 'gamma': 0.1915704647548995, 'kernel': 'poly', 'degree': 7}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:01:37,729] Trial 3 finished with value: 0.0 and parameters: {'C': 1.4809892204552142, 'gamma': 0.29922320204986597, 'kernel': 'rbf', 'degree': 6}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-0

[I 2023-07-02 23:02:04,155] Trial 37 finished with value: 0.8069975928010802 and parameters: {'C': 1.0620472883306085, 'gamma': 0.618857154432178, 'kernel': 'linear', 'degree': 8}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:02:04,563] Trial 38 finished with value: 0.3712287317486988 and parameters: {'C': 7.000455835853153, 'gamma': 0.7054592431472382, 'kernel': 'sigmoid', 'degree': 9}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:02:05,489] Trial 39 finished with value: 0.13548239629705144 and parameters: {'C': 9.141081470309066, 'gamma': 0.5162289748723284, 'kernel': 'poly', 'degree': 9}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:02:06,600] Trial 40 finished with value: 0.0 and parameters: {'C': 8.911052883993907, 'gamma': 0.3446152052830205, 'kernel': 'rbf', 'degree': 6}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:02:07,013] Trial 41 finished with value: 0.3756664300458602 and parameters: {'C': 5.4

[I 2023-07-02 23:02:25,792] Trial 74 finished with value: 0.0 and parameters: {'C': 9.87403368021795, 'gamma': 0.15891272219249292, 'kernel': 'rbf', 'degree': 9}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:02:26,909] Trial 75 finished with value: 0.0 and parameters: {'C': 4.740062281970205, 'gamma': 0.42067130731428853, 'kernel': 'rbf', 'degree': 10}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:02:27,825] Trial 76 finished with value: 0.2629352418578247 and parameters: {'C': 9.966704687031664, 'gamma': 0.5598773885466012, 'kernel': 'poly', 'degree': 6}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:02:28,975] Trial 77 finished with value: 0.0 and parameters: {'C': 1.3786782099998005, 'gamma': 0.9545105169861352, 'kernel': 'rbf', 'degree': 5}. Best is trial 0 with value: 0.8069975928010802.
[I 2023-07-02 23:02:29,386] Trial 78 finished with value: 0.3797040132785906 and parameters: {'C': 1.2242201627763274, 'gamma': 0.6748574

In [33]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [34]:
best_params

{'C': 3.807947176588889,
 'gamma': 0.951207163345817,
 'kernel': 'linear',
 'degree': 2}

In [36]:
best_features

array([ True, False,  True, ..., False, False, False])

#### Get best parameters and make predictions

In [37]:
selected_features_indices = [i for i, value in enumerate(best_features) if value]
X_test_selected = X_test[:, selected_features_indices]

In [38]:
X_train_val_selected_final_features = X_train_val[:, selected_features_indices]

In [39]:
X_test_selected.shape

(286, 300)

In [40]:
best_classifier_SVC = SVC(**best_params)


In [41]:
best_classifier_SVC.fit(X_train_val_selected_final_features, y_train_val)

SVC(C=3.807947176588889, degree=2, gamma=0.951207163345817, kernel='linear')

In [42]:
y_test_pred = best_classifier_SVC.predict(X_test_selected)

In [43]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_SVC = balanced_accuracy_score(y_test, y_test_pred)
precision_SVC = precision_score(y_test, y_test_pred,average='weighted')
recall_SVC = recall_score(y_test, y_test_pred,average='weighted')
f1_SVC = f1_score(y_test, y_test_pred,average='weighted')

In [44]:
balanced_acc_SVC

0.842797884127189

In [45]:
precision_SVC

0.8606172703010647

In [46]:
recall_SVC

0.8601398601398601

In [47]:
f1_SVC

0.8588390549969841

In [48]:
y_test_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 0, 3, 1, 0, 3, 3, 2,
       0, 2, 3, 0, 3, 2, 0, 0, 3, 3, 1, 1, 0, 0, 3, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 2, 1, 2, 1, 3, 0, 1, 0, 1, 2,
       0, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 0, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 3, 3, 0, 0, 0, 1, 2, 0, 1, 1, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 0, 3, 2, 2, 3, 3, 0, 0, 0, 3, 3, 2, 0, 2, 2,
       2, 1, 0, 3, 0, 2, 0, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 0, 0, 3, 3, 1, 0, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 2, 0, 3, 0, 0, 0, 0, 1, 2])

In [49]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [50]:
# Get the coefficients and support vectors
coefs = best_classifier_SVC.coef_
support_vectors = best_classifier_SVC.support_vectors_

# Calculate the feature importance scores
feature_importance = np.abs(np.dot(coefs, support_vectors.T))

# Get the top 20 indices of the feature importance scores
top20_indices = np.argsort(feature_importance[0])[-20:]

# Print the top 20 indices
print(top20_indices)

[237 186 301 201 303  83 247 236 220 261 194 292 222  60 223 244 254  37
 136 128]


In [51]:
#select the top 20 important features of the classifier
top20_indices_SVC= pd.Series(top20_indices).sort_values(ascending=False)

# Filter the DataFrame using the boolean array
data_filtered_features =df_filtered.iloc[:,1:-9].loc[:, best_features]

# Print the filtered DataFrame
data_filtered_features

# Get the column names corresponding to the series indices
column_names_SVC = data_filtered_features.columns[top20_indices_SVC.index]

# Print the column names
print(column_names_SVC)

Index(['ENSG00000026025', 'ENSG00000087086', 'ENSG00000112306',
       'ENSG00000145425', 'ENSG00000144713', 'ENSG00000198918',
       'ENSG00000184009', 'ENSG00000205542', 'ENSG00000111640',
       'ENSG00000145423', 'ENSG00000187514', 'ENSG00000034510',
       'ENSG00000075624', 'ENSG00000147604', 'ENSG00000167996',
       'ENSG00000149273', 'ENSG00000137154', 'ENSG00000140988',
       'ENSG00000138326', 'ENSG00000134419'],
      dtype='object')


## Random Forest

#### Cross val for hyperparameter tunning and feature selection

In [52]:
# Define the number of folds for cross-validation
n_folds = 5

In [53]:
# Define the objective function for Optuna hyperparameter tuning
def objective_RF(trial):
  global best_features
  # Define the hyperparameters to be optimized
  n_estimators= trial.suggest_int('n_estimators', 100,1000)
  max_depth= trial.suggest_int('max_depth', 5, 31)
  min_samples_split= trial.suggest_int('min_samples_split', 2, 100)
  min_samples_leaf= trial.suggest_int('min_samples_leaf', 1, 4)
  bootstrap=trial.suggest_categorical('bootstrap', [True, False])

  # Instantiate the classifier with the current hyperparameters
  classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,bootstrap=bootstrap)

  # Perform cross-validation with feature selection
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []
  feature_selector = SelectKBest(score_func=f_classif, k=300)  # Adjust the number of features as desired
  best_features = None

  for train_index, val_index in kf.split(X_train_val, y_train_val):
      X_train, X_val = X_train_val[train_index], X_train_val[val_index]
      y_train, y_val = y_train_val[train_index], y_train_val[val_index]

      # Perform feature selection on the training set
      X_train_selected = feature_selector.fit_transform(X_train, y_train)
      selected_features = feature_selector.get_support()

      # Apply the same feature selection on the validation set
      X_val_selected = X_val[:, selected_features]

      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train_selected, y_train)
      y_pred = classifier.predict(X_val_selected)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)

  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  if trial.should_prune()==False:
    best_features = selected_features.copy()

  return avg_mcc

In [54]:
# Define the Optuna study
study = optuna.create_study(direction='maximize',sampler=RandomSampler(seed=42))
study.optimize(objective_RF, n_trials=100)

[I 2023-07-02 23:02:44,311] A new study created in memory with name: no-name-dcb340a6-cb6f-4dac-ba72-2a68cad3f173
[I 2023-07-02 23:03:00,754] Trial 0 finished with value: 0.8278645031523357 and parameters: {'n_estimators': 437, 'max_depth': 30, 'min_samples_split': 74, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 0 with value: 0.8278645031523357.
[I 2023-07-02 23:03:10,135] Trial 1 finished with value: 0.8356073901408898 and parameters: {'n_estimators': 152, 'max_depth': 28, 'min_samples_split': 61, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 1 with value: 0.8356073901408898.
[I 2023-07-02 23:04:01,195] Trial 2 finished with value: 0.8419249928184153 and parameters: {'n_estimators': 850, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 2 with value: 0.8419249928184153.
[I 2023-07-02 23:04:30,484] Trial 3 finished with value: 0.8369720299433926 and parameters: {'n_estimators': 489, 'max_depth': 12, 'min_samples

[I 2023-07-02 23:17:39,841] Trial 33 finished with value: 0.8368868084095828 and parameters: {'n_estimators': 899, 'max_depth': 26, 'min_samples_split': 65, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 9 with value: 0.8546689450491212.
[I 2023-07-02 23:18:04,951] Trial 34 finished with value: 0.816847448559221 and parameters: {'n_estimators': 646, 'max_depth': 5, 'min_samples_split': 12, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 9 with value: 0.8546689450491212.
[I 2023-07-02 23:18:29,829] Trial 35 finished with value: 0.8263865644639358 and parameters: {'n_estimators': 594, 'max_depth': 23, 'min_samples_split': 66, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 9 with value: 0.8546689450491212.
[I 2023-07-02 23:18:44,456] Trial 36 finished with value: 0.8252816088756987 and parameters: {'n_estimators': 393, 'max_depth': 25, 'min_samples_split': 66, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 9 with value: 0.8546689450491212.
[I 2023-

[I 2023-07-02 23:32:37,070] Trial 67 finished with value: 0.8305594983990678 and parameters: {'n_estimators': 555, 'max_depth': 27, 'min_samples_split': 33, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 9 with value: 0.8546689450491212.
[I 2023-07-02 23:33:07,429] Trial 68 finished with value: 0.8292696497851686 and parameters: {'n_estimators': 915, 'max_depth': 7, 'min_samples_split': 33, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 9 with value: 0.8546689450491212.
[I 2023-07-02 23:33:51,776] Trial 69 finished with value: 0.841929427089992 and parameters: {'n_estimators': 669, 'max_depth': 17, 'min_samples_split': 31, 'min_samples_leaf': 2, 'bootstrap': False}. Best is trial 9 with value: 0.8546689450491212.
[I 2023-07-02 23:34:48,635] Trial 70 finished with value: 0.8548188609132218 and parameters: {'n_estimators': 813, 'max_depth': 26, 'min_samples_split': 11, 'min_samples_leaf': 2, 'bootstrap': False}. Best is trial 70 with value: 0.8548188609132218.
[I 2023

In [55]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [56]:
best_params

{'n_estimators': 813,
 'max_depth': 26,
 'min_samples_split': 11,
 'min_samples_leaf': 2,
 'bootstrap': False}

In [58]:
best_features

array([ True, False,  True, ..., False, False, False])

#### Get best parameters and make predictions

In [59]:
selected_features_indices = [i for i, value in enumerate(best_features) if value]
X_test_selected = X_test[:, selected_features_indices]

In [60]:
X_train_val_selected_final_features = X_train_val[:, selected_features_indices]

In [61]:
X_test_selected.shape

(286, 300)

In [63]:
best_classifier_RF = RandomForestClassifier(**best_params)

In [65]:
best_classifier_RF.fit(X_train_val_selected_final_features, y_train_val)

RandomForestClassifier(bootstrap=False, max_depth=26, min_samples_leaf=2,
                       min_samples_split=11, n_estimators=813)

In [66]:
y_test_pred = best_classifier_RF.predict(X_test_selected)

In [67]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_RF = balanced_accuracy_score(y_test, y_test_pred)
precision_RF = precision_score(y_test, y_test_pred,average='weighted')
recall_RF = recall_score(y_test, y_test_pred,average='weighted')
f1_RF = f1_score(y_test, y_test_pred,average='weighted')

In [68]:
balanced_acc_RF

0.9015803698205758

In [69]:
precision_RF

0.9135783541420457

In [70]:
recall_RF

0.9125874125874126

In [71]:
f1_RF

0.912028607560937

In [72]:
y_test_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 1, 0, 0, 0, 3, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 0, 0, 2, 2, 0, 2, 0, 0, 0, 3, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 2, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 3, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 2, 1, 2, 0, 1, 1, 0, 3, 1, 1, 2, 0, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3, 0, 2, 0, 3, 0, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 0, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 0, 2, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [73]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [74]:
X_test_selected.shape

(286, 300)

#### Feature importance

In [75]:
# Filter the DataFrame using the boolean array
data_filtered_features = df_filtered.iloc[:,:-10].loc[:, best_features]

# Print the filtered DataFrame
data_filtered_features

,cell,ENSG00000198804,ENSG00000156508,ENSG00000087086,ENSG00000229117,ENSG00000026025,ENSG00000140988,ENSG00000198918,ENSG00000198938,ENSG00000118271,...,ENSG00000065548,ENSG00000115866,ENSG00000112715,ENSG00000159352,ENSG00000102225,ENSG00000184897,ENSG00000006652,ENSG00000276256,ENSG00000128050,ENSG00000182628
0,ERR2538859-AAACCTGAGACCACGA,9065.20200,4735.2563,5042.7400,9255.0700,3659.0615,5749.6270,3874.3005,2982.5964,0.0,...,92.245255,153.742080,30.748417,61.496834,215.238920,215.23892,61.496834,122.993670,61.496834,153.742080
1,ERR2538859-AAACCTGTCTGATACG,0.00000,16139.4430,4519.0444,12696.3620,3443.0813,7962.1255,10329.2440,0.0000,0.0,...,0.000000,215.192580,0.000000,215.192580,215.192580,0.00000,0.000000,0.000000,0.000000,215.192580
2,ERR2538859-AAACGGGAGTGTTGAA,9193.14500,13334.9840,4739.7560,14643.7230,2193.0212,12343.2730,11424.9340,3218.7893,0.0,...,141.485240,70.742620,35.371310,70.742620,0.000000,70.74262,35.371310,70.742620,141.485240,35.371310
3,ERR2538859-AAAGATGTCCGAACGC,10016.36700,10540.0990,4058.9202,7725.0415,8837.9720,7200.7144,5695.5815,2029.4601,0.0,...,196.399350,65.466450,0.000000,0.000000,0.000000,0.00000,32.733227,65.466450,0.000000,0.000000
4,ERR2538859-AAAGTAGGTTAGTGGG,18116.04300,5664.7690,5384.3350,8469.1100,3140.8620,4935.0034,4823.4670,6449.9844,0.0,...,56.086823,112.173645,168.260470,56.086823,112.173645,0.00000,0.000000,56.086823,56.086823,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,ERR2538860-TTTGGTTTCGTTACGA,11986.30100,6563.9270,5707.7627,14554.7940,1141.5525,12557.0770,6563.9270,7420.0913,0.0,...,0.000000,285.388120,0.000000,0.000000,0.000000,0.00000,0.000000,285.388120,0.000000,285.388120
1423,ERR2538860-TTTGTCAAGCCCAACC,172.20595,5510.5903,9126.9150,10160.1510,7577.0615,7232.6500,6027.2080,0.0000,0.0,...,172.205950,172.205950,0.000000,172.205950,0.000000,0.00000,172.205950,0.000000,0.000000,0.000000
1424,ERR2538860-TTTGTCACATTGGGCC,14539.58000,10060.2140,2570.1277,8003.4385,6094.8745,8224.4080,9105.5960,5140.2554,0.0,...,220.296660,367.161100,0.000000,73.432220,146.864440,0.00000,220.296660,146.864440,146.864440,73.432220
1425,ERR2538860-TTTGTCAGTTGCGTTA,5771.68550,6992.6187,11654.3640,8713.0250,6160.1640,7714.0796,4384.2610,2830.3457,0.0,...,110.993950,166.490920,110.993950,166.490920,221.987900,0.00000,221.987900,0.000000,110.993950,0.000000


In [76]:
# view the feature scores
#X_df = pd.DataFrame(X_train_val,columns=X.columns)
feature_scores = pd.Series(best_classifier_RF.feature_importances_).sort_values(ascending=False)

In [77]:
feature_scores

59     0.031908
77     0.027471
78     0.025254
286    0.024412
194    0.020526
         ...   
46     0.000427
295    0.000407
282    0.000397
197    0.000389
294    0.000387
Length: 300, dtype: float64

In [78]:
# Get the column names corresponding to the series indices
column_names = data_filtered_features.columns[feature_scores.index]

# Print the column names
print(column_names)

Index(['ENSG00000174444', 'ENSG00000132475', 'ENSG00000137309',
       'ENSG00000111011', 'ENSG00000141504', 'ENSG00000163220',
       'ENSG00000074800', 'ENSG00000167460', 'ENSG00000184840',
       'ENSG00000163864',
       ...
       'ENSG00000106211', 'ENSG00000198830', 'ENSG00000177156',
       'ENSG00000125835', 'ENSG00000135940', 'ENSG00000163191',
       'ENSG00000184897', 'ENSG00000124733', 'ENSG00000046647',
       'ENSG00000102225'],
      dtype='object', length=300)


In [79]:
# Get the column names corresponding to the series indices above the threshold
filtered_column_names = data_filtered_features.columns[feature_scores[feature_scores > 0.05].index]

# Print the filtered column names
print(filtered_column_names)

Index([], dtype='object')


In [80]:
# Get feature importance scores
importance = pd.Series(best_classifier_RF.feature_importances_).sort_values(ascending=False)

## Print feature importance scores
#print("Feature Importance:")
#for i, imp in enumerate(importance):
#    print(f"Feature {i}: {imp}")


# Filter the DataFrame using the boolean array
data_filtered_features = df_filtered.iloc[:,1:-9].loc[:, best_features]

# Print the filtered DataFrame
data_filtered_features

# Get the column names corresponding to the series indices
column_names_RF = data_filtered_features.columns[importance.index]

# Print the column names
print(column_names_RF)

# Get the column names corresponding to the series indices above the threshold
filtered_column_names_RF = data_filtered_features.columns[importance[importance > 0.01].index]

# Print the filtered column names
print(filtered_column_names_RF)

Index(['ENSG00000163453', 'ENSG00000164104', 'ENSG00000163331',
       'ENSG00000066279', 'ENSG00000131747', 'ENSG00000173212',
       'ENSG00000197061', 'ENSG00000167680', 'ENSG00000148773',
       'ENSG00000117724',
       ...
       'ENSG00000127184', 'ENSG00000196262', 'ENSG00000123989',
       'ENSG00000145335', 'ENSG00000188229', 'ENSG00000110700',
       'ENSG00000182263', 'ENSG00000160953', 'ENSG00000244398',
       'ENSG00000184897'],
      dtype='object', length=300)
Index(['ENSG00000163453', 'ENSG00000164104', 'ENSG00000163331',
       'ENSG00000066279', 'ENSG00000131747', 'ENSG00000173212',
       'ENSG00000197061', 'ENSG00000167680', 'ENSG00000148773',
       'ENSG00000117724', 'ENSG00000113810', 'ENSG00000198513',
       'ENSG00000168014', 'ENSG00000144649', 'ENSG00000225890',
       'ENSG00000184302', 'ENSG00000109046', 'ENSG00000163864',
       'ENSG00000138061', 'ENSG00000049540', 'ENSG00000115947',
       'ENSG00000175063', 'ENSG00000167996', 'ENSG00000164032',
      

In [81]:
len(filtered_column_names_RF)

28

## XGBoost

#### Cross val for hyperparameter tunning and feature selection

In [82]:
# Define the number of folds for cross-validation
n_folds = 5

In [83]:
# Define the objective function for Optuna hyperparameter tuning
def objective_XG(trial):
  global best_features

  params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_int('max_depth', 5, 20),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
            'random_state': 42,
            'tree_method': 'hist',  # Optional: Use 'hist' method for faster training
            'objective': 'multi:softmax',
            'num_class': 4  # Replace with the actual number of classes
        }

  # Instantiate the classifier with the current hyperparameters
  #classifier = XGBClassifier(eta=eta,gamma=gamma ,min_child_weight=min_child_weight,max_delta_step=max_delta_step,subsample=subsample)
  classifier = XGBClassifier(**params)

  # Perform cross-validation with feature selection
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []
  feature_selector = SelectKBest(score_func=f_classif, k=300)  # Adjust the number of features as desired
  best_features = None

  for train_index, val_index in kf.split(X_train_val, y_train_val):
      X_train, X_val = X_train_val[train_index], X_train_val[val_index]
      y_train, y_val = y_train_val[train_index], y_train_val[val_index]

      # Perform feature selection on the training set
      X_train_selected = feature_selector.fit_transform(X_train, y_train)
      selected_features = feature_selector.get_support()

      # Apply the same feature selection on the validation set
      X_val_selected = X_val[:, selected_features]

      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train_selected, y_train)
      y_pred = classifier.predict(X_val_selected)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)

  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  if trial.should_prune()==False:
    best_features = selected_features.copy()

  return avg_mcc

In [84]:
# Define the Optuna study
study = optuna.create_study(direction='maximize', sampler=RandomSampler(seed=42))
study.optimize(objective_XG, n_trials=100)

[I 2023-07-02 23:48:06,765] A new study created in memory with name: no-name-634e9cd0-5e6e-4aea-8902-edf577bac055
[I 2023-07-02 23:48:25,681] Trial 0 finished with value: 0.8560057649776948 and parameters: {'n_estimators': 106, 'max_depth': 20, 'learning_rate': 0.07587945476302646, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'gamma': 0.7799726016810132, 'min_child_weight': 1}. Best is trial 0 with value: 0.8560057649776948.
[I 2023-07-02 23:48:34,125] Trial 1 finished with value: 0.842941670915514 and parameters: {'n_estimators': 180, 'max_depth': 14, 'learning_rate': 0.0737265320016441, 'subsample': 0.5102922471479012, 'colsample_bytree': 0.9849549260809971, 'gamma': 4.162213204002109, 'min_child_weight': 2}. Best is trial 0 with value: 0.8560057649776948.
[I 2023-07-02 23:48:44,978] Trial 2 finished with value: 0.8443895950438899 and parameters: {'n_estimators': 77, 'max_depth': 7, 'learning_rate': 0.0373818018663584, 'subsample': 0.762378215816119, 'cols

[I 2023-07-02 23:53:48,494] Trial 24 finished with value: 0.8496116625810158 and parameters: {'n_estimators': 56, 'max_depth': 14, 'learning_rate': 0.07098079256580542, 'subsample': 0.508293914463928, 'colsample_bytree': 0.7560465291496405, 'gamma': 1.1324788759896898, 'min_child_weight': 4}. Best is trial 20 with value: 0.8598737841961313.
[I 2023-07-02 23:54:00,712] Trial 25 finished with value: 0.8497503130717174 and parameters: {'n_estimators': 76, 'max_depth': 16, 'learning_rate': 0.044806181167048376, 'subsample': 0.9683649943683672, 'colsample_bytree': 0.5687604720729966, 'gamma': 1.7053317552512925, 'min_child_weight': 1}. Best is trial 20 with value: 0.8598737841961313.
[I 2023-07-02 23:54:17,666] Trial 26 finished with value: 0.84957944116265 and parameters: {'n_estimators': 189, 'max_depth': 19, 'learning_rate': 0.033214746494364004, 'subsample': 0.8299920230170895, 'colsample_bytree': 0.9086111001006079, 'gamma': 2.7760040579973118, 'min_child_weight': 3}. Best is trial 20 

[I 2023-07-02 23:59:14,800] Trial 48 finished with value: 0.8545932817075548 and parameters: {'n_estimators': 179, 'max_depth': 16, 'learning_rate': 0.05267564461785927, 'subsample': 0.5489170803255008, 'colsample_bytree': 0.7458079375584161, 'gamma': 2.3673588539028283, 'min_child_weight': 1}. Best is trial 40 with value: 0.8650974263082819.
[I 2023-07-02 23:59:23,364] Trial 49 finished with value: 0.8454236751002613 and parameters: {'n_estimators': 115, 'max_depth': 11, 'learning_rate': 0.06542650882469948, 'subsample': 0.8175468254338218, 'colsample_bytree': 0.5226520048860223, 'gamma': 1.873063073132356, 'min_child_weight': 4}. Best is trial 40 with value: 0.8650974263082819.
[I 2023-07-02 23:59:30,317] Trial 50 finished with value: 0.8443887532829037 and parameters: {'n_estimators': 125, 'max_depth': 18, 'learning_rate': 0.06928242684570506, 'subsample': 0.5814672135407148, 'colsample_bytree': 0.5352843737002149, 'gamma': 3.212096391031578, 'min_child_weight': 1}. Best is trial 40

[I 2023-07-03 00:03:26,534] Trial 72 finished with value: 0.8417170089843067 and parameters: {'n_estimators': 153, 'max_depth': 7, 'learning_rate': 0.09198344660444582, 'subsample': 0.9112686214615845, 'colsample_bytree': 0.974899956645962, 'gamma': 3.6285975419418, 'min_child_weight': 4}. Best is trial 40 with value: 0.8650974263082819.
[I 2023-07-03 00:03:33,121] Trial 73 finished with value: 0.8546380750244144 and parameters: {'n_estimators': 113, 'max_depth': 19, 'learning_rate': 0.08794575005503676, 'subsample': 0.5226093350530947, 'colsample_bytree': 0.5131834872486261, 'gamma': 1.8823168343902479, 'min_child_weight': 5}. Best is trial 40 with value: 0.8650974263082819.
[I 2023-07-03 00:03:42,421] Trial 74 finished with value: 0.8455433391655184 and parameters: {'n_estimators': 199, 'max_depth': 7, 'learning_rate': 0.06347176438169216, 'subsample': 0.6904454283155108, 'colsample_bytree': 0.9849571989073016, 'gamma': 4.210594615678543, 'min_child_weight': 5}. Best is trial 40 with

[I 2023-07-03 00:08:30,926] Trial 96 finished with value: 0.8571464514185507 and parameters: {'n_estimators': 62, 'max_depth': 13, 'learning_rate': 0.04693571442906954, 'subsample': 0.9911893084543032, 'colsample_bytree': 0.5560194510840262, 'gamma': 1.9892779952287083, 'min_child_weight': 5}. Best is trial 40 with value: 0.8650974263082819.
[I 2023-07-03 00:08:41,902] Trial 97 finished with value: 0.8457247183903986 and parameters: {'n_estimators': 180, 'max_depth': 18, 'learning_rate': 0.03321125443404459, 'subsample': 0.585443793695033, 'colsample_bytree': 0.8343216099622155, 'gamma': 4.646879945637929, 'min_child_weight': 3}. Best is trial 40 with value: 0.8650974263082819.
[I 2023-07-03 00:08:50,094] Trial 98 finished with value: 0.8496280762051972 and parameters: {'n_estimators': 136, 'max_depth': 9, 'learning_rate': 0.07925436398727433, 'subsample': 0.5935218742787617, 'colsample_bytree': 0.6618396182021218, 'gamma': 2.127182193082084, 'min_child_weight': 3}. Best is trial 40 wi

In [85]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [86]:
best_params

{'n_estimators': 184,
 'max_depth': 10,
 'learning_rate': 0.04380246573759497,
 'subsample': 0.5469909699204345,
 'colsample_bytree': 0.789140070498087,
 'gamma': 0.17971136898371043,
 'min_child_weight': 3}

In [87]:
#best_params

In [88]:
best_features

array([ True, False,  True, ..., False, False, False])

In [89]:
len(best_features)

1500

#### Get best parameters and make predictions

In [90]:
selected_features_indices = [i for i, value in enumerate(best_features) if value]
X_test_selected = X_test[:, selected_features_indices]

In [91]:
X_train_val_selected_final_features = X_train_val[:, selected_features_indices]

In [92]:
X_test_selected.shape

(286, 300)

In [93]:
#best_classifier_XG = XGBClassifier(eta=0.9998923071077181,gamma=0.48062735928448386,min_child_weight=0.9165527129072861,max_delta_step=9,subsample=0.9036565255711814)
best_classifier_XG = XGBClassifier(**best_params)

In [94]:
best_classifier_XG.fit(X_train_val_selected_final_features, y_train_val)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.789140070498087, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.17971136898371043, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.04380246573759497, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=184, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [95]:
y_test_pred = best_classifier_XG.predict(X_test_selected)

In [96]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_XGB = balanced_accuracy_score(y_test, y_test_pred)
precision_XGB = precision_score(y_test, y_test_pred,average='weighted')
recall_XGB = recall_score(y_test, y_test_pred,average='weighted')
f1_XGB = f1_score(y_test, y_test_pred,average='weighted')

In [97]:
balanced_acc_XGB

0.9058870638096468

In [98]:
precision_XGB

0.9162027446434721

In [99]:
recall_XGB

0.916083916083916

In [100]:
f1_XGB

0.9154890379595907

In [101]:
y_test_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 1, 0, 0, 0, 3, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 0, 0, 2, 2, 0, 2, 0, 0, 0, 3, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 2, 0, 1, 2, 1, 0, 0, 1, 3, 3, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 3, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 2, 1, 2, 0, 1, 1, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 1, 3, 3, 0, 2, 0, 3, 0, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 1, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 0, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 2, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [102]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [103]:
# Get feature importance scores
importance = pd.Series(best_classifier_XG.feature_importances_).sort_values(ascending=False)

## Print feature importance scores
#print("Feature Importance:")
#for i, imp in enumerate(importance):
#    print(f"Feature {i}: {imp}")


# Filter the DataFrame using the boolean array
data_filtered_features = df_filtered.iloc[:,1:-9].loc[:, best_features]

# Print the filtered DataFrame
data_filtered_features

# Get the column names corresponding to the series indices
column_names_XG = data_filtered_features.columns[importance.index]

# Print the column names
print(column_names_XG)

# Get the column names corresponding to the series indices above the threshold
filtered_column_names_XG = data_filtered_features.columns[importance[importance > 0.01].index]

# Print the filtered column names
print(filtered_column_names_XG)

Index(['ENSG00000173212', 'ENSG00000167680', 'ENSG00000148773',
       'ENSG00000198513', 'ENSG00000163453', 'ENSG00000140279',
       'ENSG00000138061', 'ENSG00000164104', 'ENSG00000066279',
       'ENSG00000197563',
       ...
       'ENSG00000083845', 'ENSG00000122406', 'ENSG00000182481',
       'ENSG00000123416', 'ENSG00000173113', 'ENSG00000280858',
       'ENSG00000149591', 'ENSG00000166441', 'ENSG00000101335',
       'ENSG00000188229'],
      dtype='object', length=300)
Index(['ENSG00000173212', 'ENSG00000167680', 'ENSG00000148773',
       'ENSG00000198513', 'ENSG00000163453', 'ENSG00000140279',
       'ENSG00000138061', 'ENSG00000164104', 'ENSG00000066279',
       'ENSG00000197563', 'ENSG00000164434', 'ENSG00000153944'],
      dtype='object')


In [104]:
np.shape(filtered_column_names_XG)

(12,)

## Logistic Regression

#### Cross val for hyperparameter tunning and feature selection

In [105]:
# Define the number of folds for cross-validation
n_folds = 5

In [106]:
# Define the objective function for Optuna hyperparameter tuning
def objective_LG(trial):
  global best_features
  params = {
              'tol' : trial.suggest_float('tol' , 1e-6 , 1e-3),
              'C' : trial.suggest_float("C", 1e-5, 100),
              "n_jobs" : -1,
              'solver': trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
              "max_iter" : 150
          }

  # Instantiate the classifier with the current hyperparameters
  #classifier = XGBClassifier(eta=eta,gamma=gamma ,min_child_weight=min_child_weight,max_delta_step=max_delta_step,subsample=subsample)
  classifier = LogisticRegression(**params)

  # Perform cross-validation with feature selection
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []
  feature_selector = SelectKBest(score_func=f_classif, k=300)  # Adjust the number of features as desired
  best_features = None

  for train_index, val_index in kf.split(X_train_val, y_train_val):
      X_train, X_val = X_train_val[train_index], X_train_val[val_index]
      y_train, y_val = y_train_val[train_index], y_train_val[val_index]

      # Perform feature selection on the training set
      X_train_selected = feature_selector.fit_transform(X_train, y_train)
      selected_features = feature_selector.get_support()

      # Apply the same feature selection on the validation set
      X_val_selected = X_val[:, selected_features]

      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train_selected, y_train)
      y_pred = classifier.predict(X_val_selected)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)

  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  if trial.should_prune()==False:
    best_features = selected_features.copy()

  return avg_mcc

In [107]:
# Define the Optuna study
study = optuna.create_study(direction='maximize', sampler=RandomSampler(seed=42))
study.optimize(objective_LG, n_trials=100)

[I 2023-07-03 00:09:04,211] A new study created in memory with name: no-name-a4cef711-e636-46a8-a040-9e76c2b8786e
[I 2023-07-03 00:09:09,589] Trial 0 finished with value: 0.8006500822092771 and parameters: {'tol': 0.0003751655787285152, 'C': 95.07143113384855, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.8006500822092771.
[I 2023-07-03 00:09:11,714] Trial 1 finished with value: 0.7478512664976665 and parameters: {'tol': 0.0008663099696291604, 'C': 60.11150516317076, 'solver': 'liblinear'}. Best is trial 0 with value: 0.8006500822092771.
[I 2023-07-03 00:09:16,385] Trial 2 finished with value: 0.8345240278650486 and parameters: {'tol': 0.00018264314223989352, 'C': 18.340459151298283, 'solver': 'saga'}. Best is trial 2 with value: 0.8345240278650486.
[I 2023-07-03 00:09:19,018] Trial 3 finished with value: 0.7534050657934289 and parameters: {'tol': 0.0001403543667913898, 'C': 29.21447193207533, 'solver': 'liblinear'}. Best is trial 2 with value: 0.8345240278650486.
[I 2023-07-03

[I 2023-07-03 00:10:38,504] Trial 21 finished with value: 0.8128478325508055 and parameters: {'tol': 0.0005031763442056327, 'C': 5.147884610211422, 'solver': 'lbfgs'}. Best is trial 19 with value: 0.8371289343751988.
[I 2023-07-03 00:10:44,188] Trial 22 finished with value: 0.8055977332311361 and parameters: {'tol': 0.0009856648036564901, 'C': 24.205534730597325, 'solver': 'lbfgs'}. Best is trial 19 with value: 0.8371289343751988.
[I 2023-07-03 00:10:46,319] Trial 23 finished with value: 0.7466348618664447 and parameters: {'tol': 0.000632673524762986, 'C': 63.35297474079236, 'solver': 'liblinear'}. Best is trial 19 with value: 0.8371289343751988.
[I 2023-07-03 00:10:47,818] Trial 24 finished with value: 0.8018873576249247 and parameters: {'tol': 4.173436641320915e-05, 'C': 59.08929840989475, 'solver': 'newton-cg'}. Best is trial 19 with value: 0.8371289343751988.
[I 2023-07-03 00:10:53,615] Trial 25 finished with value: 0.8018873576249247 and parameters: {'tol': 0.00017519206257598648,

[I 2023-07-03 00:12:33,106] Trial 59 finished with value: 0.8371898160993622 and parameters: {'tol': 0.000573864450235163, 'C': 63.18372489860781, 'solver': 'saga'}. Best is trial 58 with value: 0.8380502160492306.
[I 2023-07-03 00:12:36,435] Trial 60 finished with value: 0.8330445067403771 and parameters: {'tol': 0.0007917874646821228, 'C': 78.96181638327396, 'solver': 'sag'}. Best is trial 58 with value: 0.8380502160492306.
[I 2023-07-03 00:12:38,647] Trial 61 finished with value: 0.752146462773599 and parameters: {'tol': 0.0008878164785755415, 'C': 35.091507746057744, 'solver': 'liblinear'}. Best is trial 58 with value: 0.8380502160492306.
/home/marina/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refe

[I 2023-07-03 00:14:38,668] Trial 95 finished with value: 0.8346435266343969 and parameters: {'tol': 0.0001537062800451477, 'C': 24.595780378873528, 'solver': 'saga'}. Best is trial 58 with value: 0.8380502160492306.
/home/marina/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2023-07-03 00:14:44,528] Trial 96 finished with value: 0.8031093292783635 and parameters: {'tol': 8.115351191598057e-05, 'C': 52.45114371191157, 'solver': 'lbfgs'}. Best is trial 58 with value: 0.8380502160492306.
[I 2023-07-03 00:14:47,850] Trial 97 finish

In [108]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [109]:
best_params

{'tol': 0.00038981247705542903, 'C': 1.0837750396533212, 'solver': 'saga'}

In [110]:
best_features

array([ True, False,  True, ..., False, False, False])

#### Get best parameters and make predictions

In [111]:
selected_features_indices = [i for i, value in enumerate(best_features) if value]
X_test_selected = X_test[:, selected_features_indices]

In [112]:
X_train_val_selected_final_features = X_train_val[:, selected_features_indices]

In [113]:
X_test_selected.shape

(286, 300)

In [114]:
#best_classifier_XG = XGBClassifier(eta=0.9998923071077181,gamma=0.48062735928448386,min_child_weight=0.9165527129072861,max_delta_step=9,subsample=0.9036565255711814)
best_classifier_LG = LogisticRegression(**best_params)

In [115]:
best_classifier_LG.fit(X_train_val_selected_final_features, y_train_val)

LogisticRegression(C=1.0837750396533212, solver='saga',
                   tol=0.00038981247705542903)

In [116]:
y_test_pred = best_classifier_LG.predict(X_test_selected)

In [117]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_LR= balanced_accuracy_score(y_test, y_test_pred)
precision_LR = precision_score(y_test, y_test_pred,average='weighted')
recall_LR = recall_score(y_test, y_test_pred,average='weighted')
f1_LR = f1_score(y_test, y_test_pred,average='weighted')

In [118]:
balanced_acc_LR

0.8804665246762188

In [119]:
precision_LR

0.8914441024562483

In [120]:
recall_LR

0.8916083916083916

In [121]:
f1_LR

0.8911806464799867

In [122]:
y_test_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 2, 2, 1, 1, 3, 3, 1, 0, 3, 3, 0,
       0, 0, 3, 0, 3, 2, 0, 0, 3, 3, 1, 1, 0, 0, 3, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 2, 1, 2, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 3, 2, 0, 1, 0, 2, 3, 2, 0, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 3, 3, 0, 0, 2, 1, 2, 0, 1, 1, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 2, 2, 3, 3, 0, 2, 0, 3, 3, 2, 0, 2, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 1, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 0, 0, 3, 3, 1, 0, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 1, 0, 3, 1, 2, 2, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [123]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [124]:
np.shape(best_classifier_LG.coef_)

(4, 300)

In [125]:
#select the top 20 important features of the classifier
top20_indices=list()
coefs=best_classifier_LG.coef_
for i in range(0,np.shape(coefs)[0]):
    top20_indices.append(np.argsort(coefs[i])[-20:])
    print(top20_indices)
    
top20_indices0=top20_indices[0]
top20_indices1=top20_indices[1]
top20_indices2=top20_indices[2]
top20_indices3=top20_indices[3]

column_names0=list()
column_names1=list()
column_names2=list()
column_names3=list()

for i in range(20):
    column_names0.append(data_filtered_features.columns[top20_indices0[i]])
    column_names1.append(data_filtered_features.columns[top20_indices1[i]])
    column_names2.append(data_filtered_features.columns[top20_indices2[i]])
    column_names3.append(data_filtered_features.columns[top20_indices3[i]])

# Print the column names
print("Important Genes for class 1:",column_names0)
print("Important Genes for class 2:",column_names1)
print("Important Genes for class 3:",column_names2)
print("Important Genes for class 4:",column_names3)



[array([ 86, 134,  53, 210, 264, 135,   8,  76, 279, 227,   0, 180, 259,
        20, 220, 287,  12, 100, 213,  62])]
[array([ 86, 134,  53, 210, 264, 135,   8,  76, 279, 227,   0, 180, 259,
        20, 220, 287,  12, 100, 213,  62]), array([ 70, 206, 169,  84,  64,  45, 179, 194,  63, 161,  42,  47, 199,
        13, 197,  43,  94, 191, 115, 126])]
[array([ 86, 134,  53, 210, 264, 135,   8,  76, 279, 227,   0, 180, 259,
        20, 220, 287,  12, 100, 213,  62]), array([ 70, 206, 169,  84,  64,  45, 179, 194,  63, 161,  42,  47, 199,
        13, 197,  43,  94, 191, 115, 126]), array([298, 167, 266, 256, 297, 182, 141, 285, 223, 205, 247, 156, 190,
       257, 216, 131, 170, 286,  32, 228])]
[array([ 86, 134,  53, 210, 264, 135,   8,  76, 279, 227,   0, 180, 259,
        20, 220, 287,  12, 100, 213,  62]), array([ 70, 206, 169,  84,  64,  45, 179, 194,  63, 161,  42,  47, 199,
        13, 197,  43,  94, 191, 115, 126]), array([298, 167, 266, 256, 297, 182, 141, 285, 223, 205, 247, 156, 1

## GaussianNB

#### Cross val for hyperparameter tunning and feature selection

In [126]:
# Define the number of folds for cross-validation
n_folds = 5

In [127]:
# Define the objective function for Optuna hyperparameter tuning
def objective_GNB(trial):
  global best_features
  params = {
            'var_smoothing': trial.suggest_float('var_smoothing', 1e-10, 1e-3, log=True)
          }

  # Instantiate the classifier with the current hyperparameters
  #classifier = XGBClassifier(eta=eta,gamma=gamma ,min_child_weight=min_child_weight,max_delta_step=max_delta_step,subsample=subsample)
  classifier = GaussianNB(**params)

  # Perform cross-validation with feature selection
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []
  feature_selector = SelectKBest(score_func=f_classif, k=300)  # Adjust the number of features as desired
  best_features = None

  for train_index, val_index in kf.split(X_train_val, y_train_val):
      X_train, X_val = X_train_val[train_index], X_train_val[val_index]
      y_train, y_val = y_train_val[train_index], y_train_val[val_index]

      # Perform feature selection on the training set
      X_train_selected = feature_selector.fit_transform(X_train, y_train)
      selected_features = feature_selector.get_support()

      # Apply the same feature selection on the validation set
      X_val_selected = X_val[:, selected_features]

      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train_selected, y_train)
      y_pred = classifier.predict(X_val_selected)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)

  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  if trial.should_prune()==False:
    best_features = selected_features.copy()

  return avg_mcc

In [128]:
# Define the Optuna study
study = optuna.create_study(direction='maximize', sampler=RandomSampler(seed=42))
study.optimize(objective_GNB, n_trials=100)

[I 2023-07-03 00:14:51,689] A new study created in memory with name: no-name-18fb86f6-e2e1-4719-8914-9c5fb0457794
[I 2023-07-03 00:14:51,814] Trial 0 finished with value: 0.7259420468885343 and parameters: {'var_smoothing': 4.185822729546966e-08}. Best is trial 0 with value: 0.7259420468885343.
[I 2023-07-03 00:14:51,920] Trial 1 finished with value: 0.7682738724400647 and parameters: {'var_smoothing': 0.0004518560951024107}. Best is trial 1 with value: 0.7682738724400647.
[I 2023-07-03 00:14:52,023] Trial 2 finished with value: 0.7558929450625923 and parameters: {'var_smoothing': 1.3303245101522907e-05}. Best is trial 1 with value: 0.7682738724400647.
[I 2023-07-03 00:14:52,126] Trial 3 finished with value: 0.7443542464200641 and parameters: {'var_smoothing': 1.5509913987594307e-06}. Best is trial 1 with value: 0.7682738724400647.
[I 2023-07-03 00:14:52,237] Trial 4 finished with value: 0.7059538885176464 and parameters: {'var_smoothing': 1.2363188277052218e-09}. Best is trial 1 with 

[I 2023-07-03 00:14:56,471] Trial 45 finished with value: 0.7478376892561618 and parameters: {'var_smoothing': 4.341661800361736e-06}. Best is trial 43 with value: 0.768797314267913.
[I 2023-07-03 00:14:56,573] Trial 46 finished with value: 0.7209198773720541 and parameters: {'var_smoothing': 1.520468869219889e-08}. Best is trial 43 with value: 0.768797314267913.
[I 2023-07-03 00:14:56,676] Trial 47 finished with value: 0.7310765810403224 and parameters: {'var_smoothing': 4.369946783595579e-07}. Best is trial 43 with value: 0.768797314267913.
[I 2023-07-03 00:14:56,780] Trial 48 finished with value: 0.7361814958183797 and parameters: {'var_smoothing': 6.713854967599219e-07}. Best is trial 43 with value: 0.768797314267913.
[I 2023-07-03 00:14:56,883] Trial 49 finished with value: 0.7059538885176464 and parameters: {'var_smoothing': 1.9678010532114953e-09}. Best is trial 43 with value: 0.768797314267913.
[I 2023-07-03 00:14:56,985] Trial 50 finished with value: 0.7682738724400647 and par

[I 2023-07-03 00:15:01,109] Trial 90 finished with value: 0.7072519730937119 and parameters: {'var_smoothing': 6.873211713642716e-10}. Best is trial 69 with value: 0.7704805369822001.
[I 2023-07-03 00:15:01,214] Trial 91 finished with value: 0.7531546292544287 and parameters: {'var_smoothing': 9.833622008382918e-06}. Best is trial 69 with value: 0.7704805369822001.
[I 2023-07-03 00:15:01,316] Trial 92 finished with value: 0.7602325515348652 and parameters: {'var_smoothing': 2.1159009829538346e-05}. Best is trial 69 with value: 0.7704805369822001.
[I 2023-07-03 00:15:01,419] Trial 93 finished with value: 0.7372036560135655 and parameters: {'var_smoothing': 8.490639132761158e-07}. Best is trial 69 with value: 0.7704805369822001.
[I 2023-07-03 00:15:01,523] Trial 94 finished with value: 0.7602325515348652 and parameters: {'var_smoothing': 2.4932754437140142e-05}. Best is trial 69 with value: 0.7704805369822001.
[I 2023-07-03 00:15:01,627] Trial 95 finished with value: 0.7313379654730584 a

In [129]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [130]:
best_params

{'var_smoothing': 0.0008094845352286139}

In [131]:
best_features

array([ True, False,  True, ..., False, False, False])

#### Get best parameters and make predictions

In [132]:
selected_features_indices = [i for i, value in enumerate(best_features) if value]
X_test_selected = X_test[:, selected_features_indices]

In [133]:
X_train_val_selected_final_features = X_train_val[:, selected_features_indices]

In [134]:
X_test_selected.shape

(286, 300)

In [135]:
#best_classifier_XG = XGBClassifier(eta=0.9998923071077181,gamma=0.48062735928448386,min_child_weight=0.9165527129072861,max_delta_step=9,subsample=0.9036565255711814)
best_classifier_GNB = GaussianNB(**best_params)

In [136]:
best_classifier_GNB.fit(X_train_val_selected_final_features, y_train_val)

GaussianNB(var_smoothing=0.0008094845352286139)

In [137]:
y_test_pred = best_classifier_GNB.predict(X_test_selected)

In [138]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_GNB= balanced_accuracy_score(y_test, y_test_pred)
precision_GNB = precision_score(y_test, y_test_pred,average='weighted')
recall_GNB = recall_score(y_test, y_test_pred,average='weighted')
f1_GNB = f1_score(y_test, y_test_pred,average='weighted')

In [139]:
balanced_acc_GNB

0.8946032066176154

In [140]:
precision_GNB

0.888048257058469

In [141]:
recall_GNB

0.8811188811188811

In [142]:
f1_GNB

0.8818557304062268

In [143]:
y_test_pred

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 3, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 2,
       0, 0, 3, 0, 1, 2, 0, 2, 3, 3, 1, 0, 2, 0, 3, 3, 2, 1, 3, 3, 3, 3,
       0, 2, 3, 0, 2, 2, 2, 0, 2, 0, 0, 0, 3, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 2, 0, 1, 0, 1, 3, 0, 0, 2, 2, 2, 0, 2, 1, 3, 0, 1, 3, 1, 1,
       2, 2, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 3, 3, 0, 0, 2, 1, 2, 0, 1, 1, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 1, 3, 3, 2, 2, 0, 3, 3, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 3, 1, 0, 0, 0, 3, 2, 1, 1, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 2, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 3, 0, 3, 1, 3, 2, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 2, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 2, 0, 3, 0, 0, 0, 0, 1, 2])

In [144]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

# MRMR for feature selection

## SVC



#### Cross val for hyperparameter tunning and feature selection

In [145]:
# Define the number of folds for cross-validation
n_folds = 5

In [148]:
def perform_feature_selection(X_train_val, y_train_val):
    X_train_features, X_hyperparam_opt, y_train_features, y_hyperparam_opt = train_test_split(X_train_val, y_train_val, train_size=0.30, random_state=42)
    # Perform feature selection on X_train_val
    X_train_val_df = pd.DataFrame(X_train_features)
    y_train_val_df = pd.DataFrame(y_train_features)
    X_hyperparam_opt = pd.DataFrame(X_hyperparam_opt)
    y_hyperparam_opt = pd.DataFrame(y_hyperparam_opt)
    selected_features = mrmr_classif(X_train_val_df, y_train_val_df, K=300)
    X_train_val_selected = X_hyperparam_opt.loc[:, selected_features]

    return X_train_val_selected, y_hyperparam_opt,selected_features

In [149]:
def objective_SVC(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features):
    # Define the hyperparameters to be optimized
    C = trial.suggest_float('C', 0.1, 10)
    gamma = trial.suggest_float('gamma', 0.01, 1)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    degree = trial.suggest_int('degree', 2, 10)

    # Instantiate the classifier with the current hyperparameters
    classifier = SVC(C=C, gamma=gamma, kernel=kernel, degree=degree)

    # Perform cross-validation on the selected features
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    cv_scores = []

    for train_index, val_index in kf.split(X_hyperparam_opt, y_hyperparam_opt):
        #X_train, X_val = X_hyperparam_opt[train_index], X_hyperparam_opt[val_index]
        #y_train, y_val = y_hyperparam_opt[train_index], y_hyperparam_opt[val_index]
        X_train, X_val = X_hyperparam_opt.iloc[train_index], X_hyperparam_opt.iloc[val_index]
        y_train, y_val = y_hyperparam_opt.iloc[train_index], y_hyperparam_opt.iloc[val_index]

        # Fit the classifier on the selected features and evaluate on the validation set
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_val)
        mcc = matthews_corrcoef(y_val, y_pred)
        cv_scores.append(mcc)

    # Calculate the average MCC
    avg_mcc = np.mean(cv_scores)

    return avg_mcc

In [150]:
# Perform feature selection
X_hyperparam_opt, y_hyperparam_opt,selected_features = perform_feature_selection(X_train_val, y_train_val)

100%|█████████████████████████████████████████| 300/300 [00:58<00:00,  5.13it/s]


In [151]:
# Define the objective function with selected features
def objective(trial):
    return objective_SVC(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features)

In [152]:
study = optuna.create_study(direction='maximize', sampler=RandomSampler(seed=42))
study.optimize(objective, n_trials=100)

[I 2023-07-03 00:16:02,143] A new study created in memory with name: no-name-c1ab909b-6867-4c1e-9eb0-293e37fe4d4b
[I 2023-07-03 00:16:02,836] Trial 0 finished with value: 0.8301557198787085 and parameters: {'C': 3.807947176588889, 'gamma': 0.951207163345817, 'kernel': 'linear', 'degree': 2}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:16:04,524] Trial 1 finished with value: 0.0 and parameters: {'C': 8.675143843171858, 'gamma': 0.6051038616257767, 'kernel': 'rbf', 'degree': 3}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:16:05,889] Trial 2 finished with value: 0.09064373854007914 and parameters: {'C': 1.9000671753502962, 'gamma': 0.1915704647548995, 'kernel': 'poly', 'degree': 7}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:16:07,525] Trial 3 finished with value: 0.0 and parameters: {'C': 1.4809892204552142, 'gamma': 0.29922320204986597, 'kernel': 'rbf', 'degree': 6}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-0

[I 2023-07-03 00:16:49,562] Trial 37 finished with value: 0.8301557198787085 and parameters: {'C': 1.0620472883306085, 'gamma': 0.618857154432178, 'kernel': 'linear', 'degree': 8}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:16:50,126] Trial 38 finished with value: 0.31114772412793207 and parameters: {'C': 7.000455835853153, 'gamma': 0.7054592431472382, 'kernel': 'sigmoid', 'degree': 9}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:16:51,484] Trial 39 finished with value: 0.02677366216344773 and parameters: {'C': 9.141081470309066, 'gamma': 0.5162289748723284, 'kernel': 'poly', 'degree': 9}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:16:53,125] Trial 40 finished with value: 0.0 and parameters: {'C': 8.911052883993907, 'gamma': 0.3446152052830205, 'kernel': 'rbf', 'degree': 6}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:16:53,672] Trial 41 finished with value: 0.3202027289102176 and parameters: {'C': 5.

[I 2023-07-03 00:17:23,520] Trial 74 finished with value: 0.0 and parameters: {'C': 9.87403368021795, 'gamma': 0.15891272219249292, 'kernel': 'rbf', 'degree': 9}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:17:25,155] Trial 75 finished with value: 0.0 and parameters: {'C': 4.740062281970205, 'gamma': 0.42067130731428853, 'kernel': 'rbf', 'degree': 10}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:17:26,485] Trial 76 finished with value: 0.1581327757683227 and parameters: {'C': 9.966704687031664, 'gamma': 0.5598773885466012, 'kernel': 'poly', 'degree': 6}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:17:28,097] Trial 77 finished with value: 0.0 and parameters: {'C': 1.3786782099998005, 'gamma': 0.9545105169861352, 'kernel': 'rbf', 'degree': 5}. Best is trial 0 with value: 0.8301557198787085.
[I 2023-07-03 00:17:28,654] Trial 78 finished with value: 0.32313326971007994 and parameters: {'C': 1.2242201627763274, 'gamma': 0.674857

In [153]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [154]:
best_params #42 random seed

{'C': 3.807947176588889,
 'gamma': 0.951207163345817,
 'kernel': 'linear',
 'degree': 2}

#### Get best parameters and make predictions

In [155]:
X_test_selected.shape

(286, 300)

In [156]:
X_train_val_selected_final_features = X_train_val[:, selected_features]

In [157]:
#best_C = study.best_params['C']
#best_gamma = study.best_params['gamma']
#best_gamma = study.best_params['gamma']
#best_classifier = SVC(C=best_C, gamma=best_gamma)
best_classifier_SVC = SVC(**best_params)

In [158]:
best_classifier_SVC.fit(X_train_val_selected_final_features, y_train_val)

SVC(C=3.807947176588889, degree=2, gamma=0.951207163345817, kernel='linear')

In [159]:
y_test_pred = best_classifier_SVC.predict(X_test_selected)

In [160]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_mrmr_SVC= balanced_accuracy_score(y_test, y_test_pred)
precision_mrmr_SVC = precision_score(y_test, y_test_pred,average='weighted')
recall_mrmr_SVC = recall_score(y_test, y_test_pred,average='weighted')
f1_mrmr_SVC = f1_score(y_test, y_test_pred,average='weighted')

In [161]:
balanced_acc_mrmr_SVC

0.25372402220214274

In [162]:
precision_mrmr_SVC

0.2832274805613966

In [163]:
recall_mrmr_SVC

0.3741258741258741

In [164]:
f1_mrmr_SVC

0.307864006647419

In [165]:
y_test_pred

array([1, 3, 0, 0, 1, 0, 3, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 2, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 1, 0, 1, 3, 0, 2, 0, 0, 2,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 2, 2, 1, 2, 2, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 1, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1])

In [166]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [167]:
# Get the coefficients and support vectors
coefs = best_classifier_SVC.coef_
support_vectors = best_classifier_SVC.support_vectors_

# Calculate the feature importance scores
feature_importance = np.abs(np.dot(coefs, support_vectors.T))

# Get the top 20 indices of the feature importance scores
top20_indices = np.argsort(feature_importance[0])[-20:]
# Get the top 20 indices of the feature importance scores
#top20_indices = np.argpartition(feature_importance, -20)[-20:]

# Print the top 20 indices
print(top20_indices)

[283 299 182 276 257  62 471  25 288 260 281 251 233 178 316 215 147 255
 266 138]


In [168]:
#select the top 20 important features of the classifier
top20_indices_SVC= pd.Series(top20_indices).sort_values(ascending=False)

# Filter the DataFrame using the boolean array
data_filtered_features =df_filtered.iloc[:,1:-9].loc[:, best_features]

# Print the filtered DataFrame
data_filtered_features

# Get the column names corresponding to the series indices
column_names_SVC = data_filtered_features.columns[top20_indices_SVC.index]

# Print the column names
print(column_names_SVC)

Index(['ENSG00000198918', 'ENSG00000145423', 'ENSG00000167996',
       'ENSG00000034510', 'ENSG00000205542', 'ENSG00000147604',
       'ENSG00000075624', 'ENSG00000149273', 'ENSG00000145425',
       'ENSG00000026025', 'ENSG00000134419', 'ENSG00000112306',
       'ENSG00000187514', 'ENSG00000184009', 'ENSG00000087086',
       'ENSG00000138326', 'ENSG00000144713', 'ENSG00000137154',
       'ENSG00000140988', 'ENSG00000111640'],
      dtype='object')


## Random Forest

#### Cross val for hyperparameter tunning and feature selection

In [169]:
# Define the number of folds for cross-validation
n_folds = 5
#n_folds_features = 2

In [170]:
def perform_feature_selection(X_train_val, y_train_val):
    X_train_features, X_hyperparam_opt, y_train_features, y_hyperparam_opt = train_test_split(X_train_val, y_train_val, train_size=0.30, random_state=42)
    # Perform feature selection on X_train_val
    X_train_val_df = pd.DataFrame(X_train_features)
    y_train_val_df = pd.DataFrame(y_train_features)
    X_hyperparam_opt = pd.DataFrame(X_hyperparam_opt)
    y_hyperparam_opt = pd.DataFrame(y_hyperparam_opt)
    selected_features = mrmr_classif(X_train_val_df, y_train_val_df, K=300)
    X_train_val_selected = X_hyperparam_opt.loc[:, selected_features]

    return X_train_val_selected, y_hyperparam_opt,selected_features

In [171]:
def objective_RF(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features):

  # Define the hyperparameters to be optimized
  n_estimators= trial.suggest_int('n_estimators', 100,1000)
  max_depth= trial.suggest_int('max_depth', 5, 31)
  min_samples_split= trial.suggest_int('min_samples_split', 2, 100)
  min_samples_leaf= trial.suggest_int('min_samples_leaf', 1, 4)
  bootstrap=trial.suggest_categorical('bootstrap', [True, False])

  # Instantiate the classifier with the current hyperparameters
  classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,bootstrap=bootstrap)

  # Perform cross-validation on the selected features
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []

  for train_index, val_index in kf.split(X_hyperparam_opt, y_hyperparam_opt):
      #X_train, X_val = X_hyperparam_opt[train_index], X_hyperparam_opt[val_index]
      #y_train, y_val = y_hyperparam_opt[train_index], y_hyperparam_opt[val_index]
      X_train, X_val = X_hyperparam_opt.iloc[train_index], X_hyperparam_opt.iloc[val_index]
      y_train, y_val = y_hyperparam_opt.iloc[train_index], y_hyperparam_opt.iloc[val_index]
      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train, y_train)
      y_pred = classifier.predict(X_val)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)
  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  return avg_mcc

In [172]:
# Perform feature selection
X_hyperparam_opt, y_hyperparam_opt,selected_features = perform_feature_selection(X_train_val, y_train_val)

100%|█████████████████████████████████████████| 300/300 [00:59<00:00,  5.07it/s]


In [173]:
# Define the objective function with selected features
def objective(trial):
    return objective_RF(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features)

In [174]:
study = optuna.create_study(direction='maximize', sampler=RandomSampler(seed=42))
study.optimize(objective, n_trials=100)

[I 2023-07-03 00:18:51,668] A new study created in memory with name: no-name-70bc71f7-6a7e-4f0f-8d2c-a36e7e114d68
[I 2023-07-03 00:19:11,053] Trial 0 finished with value: 0.8253058454474417 and parameters: {'n_estimators': 437, 'max_depth': 30, 'min_samples_split': 74, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 0 with value: 0.8253058454474417.
[I 2023-07-03 00:19:22,862] Trial 1 finished with value: 0.832625727711962 and parameters: {'n_estimators': 152, 'max_depth': 28, 'min_samples_split': 61, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 1 with value: 0.832625727711962.
[I 2023-07-03 00:20:34,057] Trial 2 finished with value: 0.8453455920857073 and parameters: {'n_estimators': 850, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 2 with value: 0.8453455920857073.
[I 2023-07-03 00:21:12,531] Trial 3 finished with value: 0.8363093073484984 and parameters: {'n_estimators': 489, 'max_depth': 12, 'min_samples_s

[I 2023-07-03 00:38:00,153] Trial 33 finished with value: 0.8419993524921248 and parameters: {'n_estimators': 899, 'max_depth': 26, 'min_samples_split': 65, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 16 with value: 0.8470784839632547.
[I 2023-07-03 00:38:35,933] Trial 34 finished with value: 0.8276048243825294 and parameters: {'n_estimators': 646, 'max_depth': 5, 'min_samples_split': 12, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 16 with value: 0.8470784839632547.
[I 2023-07-03 00:39:05,280] Trial 35 finished with value: 0.8345600619541326 and parameters: {'n_estimators': 594, 'max_depth': 23, 'min_samples_split': 66, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 16 with value: 0.8470784839632547.
[I 2023-07-03 00:39:22,862] Trial 36 finished with value: 0.8234771658892418 and parameters: {'n_estimators': 393, 'max_depth': 25, 'min_samples_split': 66, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 16 with value: 0.8470784839632547.
[I 

[I 2023-07-03 00:56:11,416] Trial 66 finished with value: 0.8308855537561273 and parameters: {'n_estimators': 873, 'max_depth': 16, 'min_samples_split': 76, 'min_samples_leaf': 4, 'bootstrap': False}. Best is trial 63 with value: 0.8490138550525389.
[I 2023-07-03 00:56:39,345] Trial 67 finished with value: 0.8384326567784489 and parameters: {'n_estimators': 555, 'max_depth': 27, 'min_samples_split': 33, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 63 with value: 0.8490138550525389.
[I 2023-07-03 00:57:19,446] Trial 68 finished with value: 0.8275818747265138 and parameters: {'n_estimators': 915, 'max_depth': 7, 'min_samples_split': 33, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 63 with value: 0.8490138550525389.
[I 2023-07-03 00:58:16,992] Trial 69 finished with value: 0.8379641479694048 and parameters: {'n_estimators': 669, 'max_depth': 17, 'min_samples_split': 31, 'min_samples_leaf': 2, 'bootstrap': False}. Best is trial 63 with value: 0.8490138550525389.
[I 

[I 2023-07-03 01:15:46,211] Trial 99 finished with value: 0.8323434720584018 and parameters: {'n_estimators': 908, 'max_depth': 17, 'min_samples_split': 68, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 63 with value: 0.8490138550525389.


In [175]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [176]:
best_params #42 random seed

{'n_estimators': 833,
 'max_depth': 12,
 'min_samples_split': 13,
 'min_samples_leaf': 3,
 'bootstrap': False}

#### Get best parameters and make predictions

In [177]:
#selected_features_indices = [i for i, value in enumerate(best_features) if value]
X_test_selected = X_test[:, selected_features]

In [178]:
X_test_selected.shape

(286, 300)

In [179]:
X_train_val_selected_final_features = X_train_val[:, selected_features]

In [180]:
X_train_val_selected_final_features.shape

(1141, 300)

In [181]:
best_classifier_RF = RandomForestClassifier(**best_params)

In [182]:
best_classifier_RF.fit(X_train_val_selected_final_features, y_train_val)

RandomForestClassifier(bootstrap=False, max_depth=12, min_samples_leaf=3,
                       min_samples_split=13, n_estimators=833)

In [183]:
y_test_pred = best_classifier_RF.predict(X_test_selected)

In [184]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_mrmr_RF= balanced_accuracy_score(y_test, y_test_pred)
precision_mrmr_RF = precision_score(y_test, y_test_pred,average='weighted')
recall_mrmr_RF = recall_score(y_test, y_test_pred,average='weighted')
f1_mrmr_RF = f1_score(y_test, y_test_pred,average='weighted')

In [185]:
balanced_acc_mrmr_RF

0.9137572070909169

In [186]:
precision_mrmr_RF

0.9241877831163545

In [187]:
recall_mrmr_RF

0.9230769230769231

In [188]:
f1_mrmr_RF

0.9224590822699559

In [189]:
y_test_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 1, 0, 0, 0, 3, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 0, 0, 2, 2, 0, 2, 0, 0, 0, 3, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 2, 1, 2, 0, 1, 1, 0, 3, 1, 1, 2, 0, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3, 0, 2, 0, 3, 0, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 0, 2, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [190]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

#### Feature importance

In [191]:
# Filter the DataFrame using the boolean array
data_filtered_features = df_filtered.iloc[:,:-10].iloc[:, selected_features]

# Print the filtered DataFrame
data_filtered_features

,ENSG00000111011,ENSG00000115947,ENSG00000166352,ENSG00000152082,ENSG00000198804,ENSG00000074800,ENSG00000109113,ENSG00000152518,ENSG00000132475,ENSG00000146540,...,ENSG00000077942,ENSG00000197006,ENSG00000139684,ENSG00000185559,ENSG00000172809,ENSG00000136888,ENSG00000270367,ENSG00000130638,ENSG00000074695,ENSG00000099622
0,122.993670,356.62660,198.68205,707.21356,9065.20200,983.94934,276.73575,276.73575,1229.93660,245.987340,...,245.987340,215.238920,30.748417,61.496834,2121.6409,338.23257,107.727104,92.245255,61.496834,399.72943
1,0.000000,0.00000,0.00000,1291.15550,0.00000,2582.31100,0.00000,215.19258,215.19258,215.192580,...,0.000000,215.192580,1075.962900,0.000000,4088.6590,215.19258,0.000000,0.000000,0.000000,860.77030
2,0.000000,353.71310,74.16356,424.45575,9193.14500,778.16880,176.85655,247.59918,353.71310,70.742620,...,0.000000,106.113940,141.485240,0.000000,3784.7302,353.71310,98.230200,35.371310,247.599180,353.71310
3,0.000000,829.24176,65.46645,392.79870,10016.36700,327.33228,196.39935,196.39935,851.06390,65.466450,...,130.932900,130.932900,130.932900,0.000000,2487.7253,327.33228,467.229550,0.000000,130.932900,785.59740
4,0.000000,612.92285,143.41394,448.69458,18116.04300,448.69458,224.34729,224.34729,785.21550,112.173645,...,56.086823,112.173645,112.173645,0.000000,2860.4280,224.34729,149.996750,56.086823,0.000000,280.43410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,0.000000,285.38812,0.00000,1426.94070,11986.30100,0.00000,0.00000,0.00000,1426.94070,285.388120,...,0.000000,0.000000,285.388120,0.000000,3995.4336,570.77625,0.000000,0.000000,0.000000,0.00000
1423,172.205950,0.00000,0.00000,0.00000,172.20595,3960.73680,0.00000,344.41190,2410.88330,0.000000,...,0.000000,0.000000,172.205950,344.411900,1205.4417,0.00000,0.000000,516.617860,0.000000,172.20595
1424,73.432220,342.68372,0.00000,220.29666,14539.58000,5140.25540,220.29666,514.02550,660.89000,146.864440,...,367.161100,0.000000,220.296660,0.000000,1909.2378,220.29666,0.000000,73.432220,367.161100,514.02550
1425,55.496975,277.48486,0.00000,776.95764,5771.68550,2719.35180,221.98790,0.00000,1720.40620,110.993950,...,0.000000,166.490920,110.993950,110.993950,1220.9335,332.98184,0.000000,0.000000,166.490920,221.98790


In [192]:
# view the feature scores
#X_df = pd.DataFrame(X_train_val,columns=X.columns)
feature_scores = pd.Series(best_classifier_RF.feature_importances_).sort_values(ascending=False)

In [193]:
feature_scores

17     0.033754
8      0.029571
69     0.027141
0      0.026865
39     0.026626
         ...   
194    0.000275
252    0.000259
251    0.000245
254    0.000236
287    0.000236
Length: 300, dtype: float64

In [194]:
# Get the column names corresponding to the series indices
column_names = data_filtered_features.columns[feature_scores.index]

# Print the column names
print(column_names)

Index(['ENSG00000174444', 'ENSG00000132475', 'ENSG00000163220',
       'ENSG00000111011', 'ENSG00000233276', 'ENSG00000137309',
       'ENSG00000204628', 'ENSG00000117519', 'ENSG00000184840',
       'ENSG00000099622',
       ...
       'ENSG00000104435', 'ENSG00000164258', 'ENSG00000077942',
       'ENSG00000171634', 'ENSG00000169139', 'ENSG00000198763',
       'ENSG00000188186', 'ENSG00000167553', 'ENSG00000106538',
       'ENSG00000115241'],
      dtype='object', length=300)


In [195]:
# Get the column names corresponding to the series indices above the threshold
filtered_column_names = data_filtered_features.columns[feature_scores[feature_scores > 0.05].index]

# Print the filtered column names
print(filtered_column_names)

Index([], dtype='object')


In [196]:
# Get feature importance scores
importance = pd.Series(best_classifier_RF.feature_importances_).sort_values(ascending=False)

# Filter the DataFrame using the boolean array
data_filtered_features = df_filtered.iloc[:,1:-9].loc[:, best_features]

# Print the filtered DataFrame
data_filtered_features

# Get the column names corresponding to the series indices
column_names_RF = data_filtered_features.columns[importance.index]

# Print the column names
print(column_names_RF)

# Get the column names corresponding to the series indices above the threshold
filtered_column_names_RF = data_filtered_features.columns[importance[importance > 0.01].index]

# Print the filtered column names
print(filtered_column_names_RF)

Index(['ENSG00000134419', 'ENSG00000034510', 'ENSG00000117394',
       'ENSG00000205542', 'ENSG00000140416', 'ENSG00000184009',
       'ENSG00000149591', 'ENSG00000282417', 'ENSG00000138326',
       'ENSG00000204899',
       ...
       'ENSG00000118596', 'ENSG00000143882', 'ENSG00000168393',
       'ENSG00000170893', 'ENSG00000117724', 'ENSG00000131747',
       'ENSG00000213949', 'ENSG00000143401', 'ENSG00000134160',
       'ENSG00000141526'],
      dtype='object', length=300)
Index(['ENSG00000134419', 'ENSG00000034510', 'ENSG00000117394',
       'ENSG00000205542', 'ENSG00000140416', 'ENSG00000184009',
       'ENSG00000149591', 'ENSG00000282417', 'ENSG00000138326',
       'ENSG00000204899', 'ENSG00000100097', 'ENSG00000173207',
       'ENSG00000144713', 'ENSG00000164104', 'ENSG00000140988',
       'ENSG00000136810', 'ENSG00000103197', 'ENSG00000137154',
       'ENSG00000142676', 'ENSG00000182718', 'ENSG00000111640',
       'ENSG00000108518', 'ENSG00000106355', 'ENSG00000026025',
      

In [197]:
len(filtered_column_names_RF)

29

## XGBoost

#### Cross val for hyperparameter tunning and feature selection

In [198]:
# Define the number of folds for cross-validation
n_folds = 5
#n_folds_features = 2

In [199]:
def perform_feature_selection(X_train_val, y_train_val):
    X_train_features, X_hyperparam_opt, y_train_features, y_hyperparam_opt = train_test_split(X_train_val, y_train_val, train_size=0.30, random_state=42)
    # Perform feature selection on X_train_val
    X_train_val_df = pd.DataFrame(X_train_features)
    y_train_val_df = pd.DataFrame(y_train_features)
    X_hyperparam_opt = pd.DataFrame(X_hyperparam_opt)
    y_hyperparam_opt = pd.DataFrame(y_hyperparam_opt)
    selected_features = mrmr_classif(X_train_val_df, y_train_val_df, K=300)
    X_train_val_selected = X_hyperparam_opt.loc[:, selected_features]

    return X_train_val_selected, y_hyperparam_opt,selected_features

In [200]:
def objective_XG(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features):

  params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_int('max_depth', 5, 20),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
            'random_state': 42,
            'tree_method': 'hist',  # Optional: Use 'hist' method for faster training
            'objective': 'multi:softmax',
            'num_class': 4  # Replace with the actual number of classes
        }

  # Instantiate the classifier with the current hyperparameters
  #classifier = XGBClassifier(eta=eta,gamma=gamma ,min_child_weight=min_child_weight,max_delta_step=max_delta_step,subsample=subsample)
  classifier = XGBClassifier(**params)

  # Perform cross-validation on the selected features
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []

  for train_index, val_index in kf.split(X_hyperparam_opt, y_hyperparam_opt):
      #X_train, X_val = X_hyperparam_opt[train_index], X_hyperparam_opt[val_index]
      #y_train, y_val = y_hyperparam_opt[train_index], y_hyperparam_opt[val_index]
      X_train, X_val = X_hyperparam_opt.iloc[train_index], X_hyperparam_opt.iloc[val_index]
      y_train, y_val = y_hyperparam_opt.iloc[train_index], y_hyperparam_opt.iloc[val_index]
      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train, y_train)
      y_pred = classifier.predict(X_val)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)
  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  return avg_mcc

In [201]:
# Perform feature selection
X_hyperparam_opt, y_hyperparam_opt,selected_features = perform_feature_selection(X_train_val, y_train_val)

100%|█████████████████████████████████████████| 300/300 [00:58<00:00,  5.12it/s]


In [202]:
# Define the objective function with selected features
def objective(trial):
    return objective_XG(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features)

In [203]:
study = optuna.create_study(direction='maximize', sampler=RandomSampler(seed=42))
study.optimize(objective, n_trials=100)

[I 2023-07-03 01:17:00,126] A new study created in memory with name: no-name-2ae40329-5e38-470e-8738-df55b5e2933e
[I 2023-07-03 01:18:02,295] Trial 0 finished with value: 0.8688409879035721 and parameters: {'n_estimators': 106, 'max_depth': 20, 'learning_rate': 0.07587945476302646, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'gamma': 0.7799726016810132, 'min_child_weight': 1}. Best is trial 0 with value: 0.8688409879035721.
[I 2023-07-03 01:18:52,691] Trial 1 finished with value: 0.8505793853294351 and parameters: {'n_estimators': 180, 'max_depth': 14, 'learning_rate': 0.0737265320016441, 'subsample': 0.5102922471479012, 'colsample_bytree': 0.9849549260809971, 'gamma': 4.162213204002109, 'min_child_weight': 2}. Best is trial 0 with value: 0.8688409879035721.
[I 2023-07-03 01:19:45,530] Trial 2 finished with value: 0.8489220055196117 and parameters: {'n_estimators': 77, 'max_depth': 7, 'learning_rate': 0.0373818018663584, 'subsample': 0.762378215816119, 'col

[I 2023-07-03 01:44:22,451] Trial 24 finished with value: 0.8578389568491339 and parameters: {'n_estimators': 56, 'max_depth': 14, 'learning_rate': 0.07098079256580542, 'subsample': 0.508293914463928, 'colsample_bytree': 0.7560465291496405, 'gamma': 1.1324788759896898, 'min_child_weight': 4}. Best is trial 20 with value: 0.8726271899587893.
[I 2023-07-03 01:45:18,604] Trial 25 finished with value: 0.8470228812748571 and parameters: {'n_estimators': 76, 'max_depth': 16, 'learning_rate': 0.044806181167048376, 'subsample': 0.9683649943683672, 'colsample_bytree': 0.5687604720729966, 'gamma': 1.7053317552512925, 'min_child_weight': 1}. Best is trial 20 with value: 0.8726271899587893.
[I 2023-07-03 01:46:45,106] Trial 26 finished with value: 0.8431875180605614 and parameters: {'n_estimators': 189, 'max_depth': 19, 'learning_rate': 0.033214746494364004, 'subsample': 0.8299920230170895, 'colsample_bytree': 0.9086111001006079, 'gamma': 2.7760040579973118, 'min_child_weight': 3}. Best is trial 2

[I 2023-07-03 02:11:06,966] Trial 48 finished with value: 0.8598792069380481 and parameters: {'n_estimators': 179, 'max_depth': 16, 'learning_rate': 0.05267564461785927, 'subsample': 0.5489170803255008, 'colsample_bytree': 0.7458079375584161, 'gamma': 2.3673588539028283, 'min_child_weight': 1}. Best is trial 20 with value: 0.8726271899587893.
[I 2023-07-03 02:11:56,328] Trial 49 finished with value: 0.8578832153299624 and parameters: {'n_estimators': 115, 'max_depth': 11, 'learning_rate': 0.06542650882469948, 'subsample': 0.8175468254338218, 'colsample_bytree': 0.5226520048860223, 'gamma': 1.873063073132356, 'min_child_weight': 4}. Best is trial 20 with value: 0.8726271899587893.
[I 2023-07-03 02:12:38,401] Trial 50 finished with value: 0.8486536057964631 and parameters: {'n_estimators': 125, 'max_depth': 18, 'learning_rate': 0.06928242684570506, 'subsample': 0.5814672135407148, 'colsample_bytree': 0.5352843737002149, 'gamma': 3.212096391031578, 'min_child_weight': 1}. Best is trial 20

[I 2023-07-03 02:33:26,147] Trial 72 finished with value: 0.8454781833025791 and parameters: {'n_estimators': 153, 'max_depth': 7, 'learning_rate': 0.09198344660444582, 'subsample': 0.9112686214615845, 'colsample_bytree': 0.974899956645962, 'gamma': 3.6285975419418, 'min_child_weight': 4}. Best is trial 20 with value: 0.8726271899587893.
[I 2023-07-03 02:34:05,837] Trial 73 finished with value: 0.8561359148657608 and parameters: {'n_estimators': 113, 'max_depth': 19, 'learning_rate': 0.08794575005503676, 'subsample': 0.5226093350530947, 'colsample_bytree': 0.5131834872486261, 'gamma': 1.8823168343902479, 'min_child_weight': 5}. Best is trial 20 with value: 0.8726271899587893.
[I 2023-07-03 02:35:03,109] Trial 74 finished with value: 0.8524332010883174 and parameters: {'n_estimators': 199, 'max_depth': 7, 'learning_rate': 0.06347176438169216, 'subsample': 0.6904454283155108, 'colsample_bytree': 0.9849571989073016, 'gamma': 4.210594615678543, 'min_child_weight': 5}. Best is trial 20 with

[I 2023-07-03 02:58:34,579] Trial 96 finished with value: 0.8525866793124701 and parameters: {'n_estimators': 62, 'max_depth': 13, 'learning_rate': 0.04693571442906954, 'subsample': 0.9911893084543032, 'colsample_bytree': 0.5560194510840262, 'gamma': 1.9892779952287083, 'min_child_weight': 5}. Best is trial 20 with value: 0.8726271899587893.
[I 2023-07-03 02:59:38,131] Trial 97 finished with value: 0.8450725487500309 and parameters: {'n_estimators': 180, 'max_depth': 18, 'learning_rate': 0.03321125443404459, 'subsample': 0.585443793695033, 'colsample_bytree': 0.8343216099622155, 'gamma': 4.646879945637929, 'min_child_weight': 3}. Best is trial 20 with value: 0.8726271899587893.
[I 2023-07-03 03:00:25,704] Trial 98 finished with value: 0.8579716537072075 and parameters: {'n_estimators': 136, 'max_depth': 9, 'learning_rate': 0.07925436398727433, 'subsample': 0.5935218742787617, 'colsample_bytree': 0.6618396182021218, 'gamma': 2.127182193082084, 'min_child_weight': 3}. Best is trial 20 wi

In [204]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [205]:
best_params #42 random seed

{'n_estimators': 195,
 'max_depth': 9,
 'learning_rate': 0.0547523655303147,
 'subsample': 0.6504391549083848,
 'colsample_bytree': 0.6424202471887338,
 'gamma': 0.18443473677266398,
 'min_child_weight': 4}

#### Get best parameters and make predictions

In [206]:
X_test_selected = X_test[:, selected_features]

In [207]:
X_test_selected.shape

(286, 300)

In [208]:
X_train_val_selected_final_features = X_train_val[:, selected_features]

In [209]:
X_train_val_selected_final_features.shape

(1141, 300)

In [210]:
best_classifier_XG = XGBClassifier(**best_params)

In [211]:
best_classifier_XG.fit(X_train_val_selected_final_features, y_train_val)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6424202471887338, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.18443473677266398, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.0547523655303147, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              n_estimators=195, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [212]:
y_test_pred = best_classifier_XG.predict(X_test_selected)

In [213]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_mrmr_XGB= balanced_accuracy_score(y_test, y_test_pred)
precision_mrmr_XGB = precision_score(y_test, y_test_pred,average='weighted')
recall_mrmr_XGB = recall_score(y_test, y_test_pred,average='weighted')
f1_mrmr_XGB= f1_score(y_test, y_test_pred,average='weighted')

In [214]:
balanced_acc_mrmr_XGB

0.9048671797685125

In [215]:
precision_mrmr_XGB

0.9167867002352319

In [216]:
recall_mrmr_XGB

0.916083916083916

In [217]:
f1_mrmr_XGB

0.9150001743868856

In [218]:
y_test_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 1, 0, 0, 0, 3, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 0, 0, 2, 2, 0, 2, 0, 0, 0, 3, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 2, 1, 0, 0, 1, 3, 3, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 2, 1, 2, 0, 1, 1, 0, 3, 1, 1, 2, 0, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 1, 3, 3, 0, 2, 0, 3, 0, 0, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 0,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 1, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 2, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [219]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [220]:
# Get feature importance scores
importance = pd.Series(best_classifier_XG.feature_importances_).sort_values(ascending=False)

## Print feature importance scores
#print("Feature Importance:")
#for i, imp in enumerate(importance):
#    print(f"Feature {i}: {imp}")


# Filter the DataFrame using the boolean array
data_filtered_features = df_filtered.iloc[:,1:-9].loc[:, best_features]

# Print the filtered DataFrame
data_filtered_features

# Get the column names corresponding to the series indices
column_names_XG = data_filtered_features.columns[importance.index]

# Print the column names
print(column_names_XG)

# Get the column names corresponding to the series indices above the threshold
filtered_column_names_XG = data_filtered_features.columns[importance[importance > 0.01].index]

# Print the filtered column names
print(filtered_column_names_XG)

Index(['ENSG00000138326', 'ENSG00000117394', 'ENSG00000140416',
       'ENSG00000134419', 'ENSG00000282417', 'ENSG00000205542',
       'ENSG00000034510', 'ENSG00000100097', 'ENSG00000142676',
       'ENSG00000168028',
       ...
       'ENSG00000197061', 'ENSG00000143882', 'ENSG00000100836',
       'ENSG00000163864', 'ENSG00000213949', 'ENSG00000100650',
       'ENSG00000188612', 'ENSG00000182263', 'ENSG00000118596',
       'ENSG00000141965'],
      dtype='object', length=300)
Index(['ENSG00000138326', 'ENSG00000117394', 'ENSG00000140416',
       'ENSG00000134419', 'ENSG00000282417', 'ENSG00000205542',
       'ENSG00000034510', 'ENSG00000100097', 'ENSG00000142676',
       'ENSG00000168028', 'ENSG00000149591'],
      dtype='object')


In [221]:
len(filtered_column_names_XG)

11

## Logistic Regression

#### Cross val for hyperparameter tunning and feature selection

In [222]:
# Define the number of folds for cross-validation
n_folds = 5

In [223]:
def perform_feature_selection(X_train_val, y_train_val):
    X_train_features, X_hyperparam_opt, y_train_features, y_hyperparam_opt = train_test_split(X_train_val, y_train_val, train_size=0.30, random_state=42)
    # Perform feature selection on X_train_val
    X_train_val_df = pd.DataFrame(X_train_features)
    y_train_val_df = pd.DataFrame(y_train_features)
    X_hyperparam_opt = pd.DataFrame(X_hyperparam_opt)
    y_hyperparam_opt = pd.DataFrame(y_hyperparam_opt)
    selected_features = mrmr_classif(X_train_val_df, y_train_val_df, K=300)
    X_train_val_selected = X_hyperparam_opt.loc[:, selected_features]

    return X_train_val_selected, y_hyperparam_opt,selected_features

In [224]:
def objective_LR(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features):

  params = {
              'tol' : trial.suggest_float('tol' , 1e-6 , 1e-3),
              'C' : trial.suggest_float("C", 1e-5, 100),
              "n_jobs" : -1,
              'solver': trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
          }

  # Instantiate the classifier with the current hyperparameters
  #classifier = XGBClassifier(eta=eta,gamma=gamma ,min_child_weight=min_child_weight,max_delta_step=max_delta_step,subsample=subsample)
  classifier = LogisticRegression(**params)

  # Perform cross-validation on the selected features
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []

  for train_index, val_index in kf.split(X_hyperparam_opt, y_hyperparam_opt):
      #X_train, X_val = X_hyperparam_opt[train_index], X_hyperparam_opt[val_index]
      #y_train, y_val = y_hyperparam_opt[train_index], y_hyperparam_opt[val_index]
      X_train, X_val = X_hyperparam_opt.iloc[train_index], X_hyperparam_opt.iloc[val_index]
      y_train, y_val = y_hyperparam_opt.iloc[train_index], y_hyperparam_opt.iloc[val_index]
      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train, y_train)
      y_pred = classifier.predict(X_val)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)
  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  return avg_mcc

In [225]:
# Perform feature selection
X_hyperparam_opt, y_hyperparam_opt,selected_features = perform_feature_selection(X_train_val, y_train_val)

100%|█████████████████████████████████████████| 300/300 [00:59<00:00,  5.04it/s]


In [226]:
# Define the objective function with selected features
def objective(trial):
    return objective_LR(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features)

In [227]:
study = optuna.create_study(direction='maximize', sampler=RandomSampler(seed=42))
study.optimize(objective, n_trials=100)

[I 2023-07-03 03:02:24,284] A new study created in memory with name: no-name-fd788c01-f943-4079-aee5-3333f052678d
[I 2023-07-03 03:02:27,094] Trial 0 finished with value: 0.8206063884982979 and parameters: {'tol': 0.0003751655787285152, 'C': 95.07143113384855, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.8206063884982979.
[I 2023-07-03 03:02:33,823] Trial 1 finished with value: 0.7868958679916298 and parameters: {'tol': 0.0008663099696291604, 'C': 60.11150516317076, 'solver': 'liblinear'}. Best is trial 0 with value: 0.8206063884982979.
[I 2023-07-03 03:02:44,161] Trial 2 finished with value: 0.8216022485507603 and parameters: {'tol': 0.00018264314223989352, 'C': 18.340459151298283, 'solver': 'saga'}. Best is trial 2 with value: 0.8216022485507603.
[I 2023-07-03 03:02:53,309] Trial 3 finished with value: 0.804697312214517 and parameters: {'tol': 0.0001403543667913898, 'C': 29.21447193207533, 'solver': 'liblinear'}. Best is trial 2 with value: 0.8216022485507603.
[I 2023-07-03 

[I 2023-07-03 03:06:48,576] Trial 38 finished with value: 0.82322967439334 and parameters: {'tol': 0.0006973187252542728, 'C': 70.24841137387008, 'solver': 'saga'}. Best is trial 16 with value: 0.8285808854344585.
[I 2023-07-03 03:06:54,821] Trial 39 finished with value: 0.8166523319079582 and parameters: {'tol': 0.0009133273120039149, 'C': 51.134244772669796, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.8285808854344585.
[I 2023-07-03 03:07:01,808] Trial 40 finished with value: 0.790362801208519 and parameters: {'tol': 0.0008901153364757489, 'C': 33.79952230520201, 'solver': 'liblinear'}. Best is trial 16 with value: 0.8285808854344585.
[I 2023-07-03 03:07:09,102] Trial 41 finished with value: 0.8252816652581532 and parameters: {'tol': 0.0005431019900728691, 'C': 28.654132347415917, 'solver': 'sag'}. Best is trial 16 with value: 0.8285808854344585.
[I 2023-07-03 03:07:12,508] Trial 42 finished with value: 0.818834148267935 and parameters: {'tol': 0.0001279334521392329, 'C': 52.2

[I 2023-07-03 03:10:58,331] Trial 76 finished with value: 0.8204877092109897 and parameters: {'tol': 0.0009966402002368315, 'C': 55.54317500594569, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.8285808854344585.
[I 2023-07-03 03:11:07,291] Trial 77 finished with value: 0.7986717939939261 and parameters: {'tol': 0.0001300302557363435, 'C': 95.40510318536197, 'solver': 'liblinear'}. Best is trial 16 with value: 0.8285808854344585.
[I 2023-07-03 03:11:14,596] Trial 78 finished with value: 0.8206604057496442 and parameters: {'tol': 0.00011444403460742941, 'C': 67.157322843548, 'solver': 'sag'}. Best is trial 16 with value: 0.8285808854344585.
[I 2023-07-03 03:11:21,858] Trial 79 finished with value: 0.81463912104334 and parameters: {'tol': 0.000561377033563851, 'C': 87.66536149929847, 'solver': 'sag'}. Best is trial 16 with value: 0.8285808854344585.
[I 2023-07-03 03:11:29,170] Trial 80 finished with value: 0.8236024846511463 and parameters: {'tol': 0.0007043756883311244, 'C': 21.2964

In [228]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [229]:
best_params #42 random seed

{'tol': 0.0009297679546902306, 'C': 80.8120398752379, 'solver': 'saga'}

#### Get best parameters and make predictions

In [230]:
#selected_features_indices = [i for i, value in enumerate(best_features) if value]
X_test_selected = X_test[:, selected_features]

In [231]:
X_test_selected.shape

(286, 300)

In [232]:
X_train_val_selected_final_features = X_train_val[:, selected_features]

In [233]:
X_train_val_selected_final_features.shape

(1141, 300)

In [234]:
#best_C = study.best_params['C']
#best_gamma = study.best_params['gamma']
#best_gamma = study.best_params['gamma']
#best_classifier = SVC(C=best_C, gamma=best_gamma)
best_classifier_LR = LogisticRegression(**best_params)

In [235]:
best_classifier_LR.fit(X_train_val_selected_final_features, y_train_val)

LogisticRegression(C=80.8120398752379, solver='saga', tol=0.0009297679546902306)

In [236]:
y_test_pred = best_classifier_LR.predict(X_test_selected)

In [237]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_mrmr_LR = balanced_accuracy_score(y_test, y_test_pred)
precision_mrmr_LR = precision_score(y_test, y_test_pred,average='weighted')
recall_mrmr_LR = recall_score(y_test, y_test_pred,average='weighted')
f1_mrmr_LR = f1_score(y_test, y_test_pred,average='weighted')

In [238]:
balanced_acc_mrmr_LR

0.8756748580095521

In [239]:
precision_mrmr_LR

0.8878278241649048

In [240]:
recall_mrmr_LR

0.8881118881118881

In [241]:
f1_mrmr_LR 

0.8875781209780631

In [242]:
y_test_pred

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 0, 3, 3, 0,
       0, 2, 3, 0, 1, 2, 0, 0, 3, 3, 1, 1, 0, 0, 0, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 2, 2, 1, 0, 1, 0, 0, 1, 3, 3, 2,
       0, 0, 1, 0, 3, 2, 0, 1, 0, 2, 3, 2, 0, 3, 0, 2, 0, 1, 3, 2, 3, 0,
       0, 2, 3, 3, 0, 0, 2, 1, 2, 1, 1, 1, 0, 3, 1, 1, 2, 1, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 0, 2, 0, 3, 3, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 1, 1, 2, 2, 0, 2, 0, 0, 3, 2, 0, 1,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 2, 0, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 3, 1, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 1, 0, 0, 1, 2])

In [243]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [244]:
#Print the top 20 important indices for each class 

top20_indices=list()
coefs=best_classifier_LG.coef_
for i in range(0,np.shape(coefs)[0]):
    top20_indices.append(np.argsort(coefs[i])[-20:])
    print(top20_indices)
    
top20_indices0=top20_indices[0]
top20_indices1=top20_indices[1]
top20_indices2=top20_indices[2]
top20_indices3=top20_indices[3]

column_names0=list()
column_names1=list()
column_names2=list()
column_names3=list()

for i in range(20):
    column_names0.append(data_filtered_features.columns[top20_indices0[i]])
    column_names1.append(data_filtered_features.columns[top20_indices1[i]])
    column_names2.append(data_filtered_features.columns[top20_indices2[i]])
    column_names3.append(data_filtered_features.columns[top20_indices3[i]])

# Print the column names
print("Important Genes for class 1:",column_names0)
print("Important Genes for class 2:",column_names1)
print("Important Genes for class 3:",column_names2)
print("Important Genes for class 4:",column_names3)


[array([ 86, 134,  53, 210, 264, 135,   8,  76, 279, 227,   0, 180, 259,
        20, 220, 287,  12, 100, 213,  62])]
[array([ 86, 134,  53, 210, 264, 135,   8,  76, 279, 227,   0, 180, 259,
        20, 220, 287,  12, 100, 213,  62]), array([ 70, 206, 169,  84,  64,  45, 179, 194,  63, 161,  42,  47, 199,
        13, 197,  43,  94, 191, 115, 126])]
[array([ 86, 134,  53, 210, 264, 135,   8,  76, 279, 227,   0, 180, 259,
        20, 220, 287,  12, 100, 213,  62]), array([ 70, 206, 169,  84,  64,  45, 179, 194,  63, 161,  42,  47, 199,
        13, 197,  43,  94, 191, 115, 126]), array([298, 167, 266, 256, 297, 182, 141, 285, 223, 205, 247, 156, 190,
       257, 216, 131, 170, 286,  32, 228])]
[array([ 86, 134,  53, 210, 264, 135,   8,  76, 279, 227,   0, 180, 259,
        20, 220, 287,  12, 100, 213,  62]), array([ 70, 206, 169,  84,  64,  45, 179, 194,  63, 161,  42,  47, 199,
        13, 197,  43,  94, 191, 115, 126]), array([298, 167, 266, 256, 297, 182, 141, 285, 223, 205, 247, 156, 1

## GaussianNB

#### Cross val for hyperparameter tunning and feature selection

In [245]:
# Define the number of folds for cross-validation
n_folds = 5
#n_folds_features = 2

In [246]:
def perform_feature_selection(X_train_val, y_train_val):
    X_train_features, X_hyperparam_opt, y_train_features, y_hyperparam_opt = train_test_split(X_train_val, y_train_val, train_size=0.30, random_state=42)
    # Perform feature selection on X_train_val
    X_train_val_df = pd.DataFrame(X_train_features)
    y_train_val_df = pd.DataFrame(y_train_features)
    X_hyperparam_opt = pd.DataFrame(X_hyperparam_opt)
    y_hyperparam_opt = pd.DataFrame(y_hyperparam_opt)
    selected_features = mrmr_classif(X_train_val_df, y_train_val_df, K=300)
    X_train_val_selected = X_hyperparam_opt.loc[:, selected_features]

    return X_train_val_selected, y_hyperparam_opt,selected_features

In [247]:
def objective_GNB(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features):

  params = {
            'var_smoothing': trial.suggest_float('var_smoothing', 1e-10, 1e-3, log=True)
          }

  # Instantiate the classifier with the current hyperparameters
  #classifier = XGBClassifier(eta=eta,gamma=gamma ,min_child_weight=min_child_weight,max_delta_step=max_delta_step,subsample=subsample)
  classifier = GaussianNB(**params)

  # Perform cross-validation on the selected features
  kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
  cv_scores = []

  for train_index, val_index in kf.split(X_hyperparam_opt, y_hyperparam_opt):
      #X_train, X_val = X_hyperparam_opt[train_index], X_hyperparam_opt[val_index]
      #y_train, y_val = y_hyperparam_opt[train_index], y_hyperparam_opt[val_index]
      X_train, X_val = X_hyperparam_opt.iloc[train_index], X_hyperparam_opt.iloc[val_index]
      y_train, y_val = y_hyperparam_opt.iloc[train_index], y_hyperparam_opt.iloc[val_index]
      # Fit the classifier on the selected features and evaluate on the validation set
      classifier.fit(X_train, y_train)
      y_pred = classifier.predict(X_val)
      mcc = matthews_corrcoef(y_val, y_pred)
      cv_scores.append(mcc)
  # Calculate the average MCC
  avg_mcc = np.mean(cv_scores)

  return avg_mcc

In [248]:
# Perform feature selection
X_hyperparam_opt, y_hyperparam_opt,selected_features = perform_feature_selection(X_train_val, y_train_val)

100%|█████████████████████████████████████████| 300/300 [00:58<00:00,  5.13it/s]


In [249]:
# Define the objective function with selected features
def objective(trial):
    return objective_GNB(trial, X_hyperparam_opt, y_hyperparam_opt,selected_features)

In [250]:
#Optimise Naive Bayes
study = optuna.create_study(direction='maximize', sampler=RandomSampler(seed=42))
study.optimize(objective, n_trials=100)

[I 2023-07-03 03:14:43,108] A new study created in memory with name: no-name-82147b9a-485a-4cd4-bfc6-afe171b9f2fc
[I 2023-07-03 03:14:43,275] Trial 0 finished with value: 0.6486412776454273 and parameters: {'var_smoothing': 4.185822729546966e-08}. Best is trial 0 with value: 0.6486412776454273.
[I 2023-07-03 03:14:43,430] Trial 1 finished with value: 0.6963049111298647 and parameters: {'var_smoothing': 0.0004518560951024107}. Best is trial 1 with value: 0.6963049111298647.
[I 2023-07-03 03:14:43,585] Trial 2 finished with value: 0.6787926283480018 and parameters: {'var_smoothing': 1.3303245101522907e-05}. Best is trial 1 with value: 0.6963049111298647.
[I 2023-07-03 03:14:43,742] Trial 3 finished with value: 0.663262072093888 and parameters: {'var_smoothing': 1.5509913987594307e-06}. Best is trial 1 with value: 0.6963049111298647.
[I 2023-07-03 03:14:43,896] Trial 4 finished with value: 0.6356777358428725 and parameters: {'var_smoothing': 1.2363188277052218e-09}. Best is trial 1 with v

[I 2023-07-03 03:14:50,147] Trial 44 finished with value: 0.6457088766199505 and parameters: {'var_smoothing': 6.478282331897327e-09}. Best is trial 11 with value: 0.6985100057396345.
[I 2023-07-03 03:14:50,302] Trial 45 finished with value: 0.6732025067793485 and parameters: {'var_smoothing': 4.341661800361736e-06}. Best is trial 11 with value: 0.6985100057396345.
[I 2023-07-03 03:14:50,458] Trial 46 finished with value: 0.6455671908551897 and parameters: {'var_smoothing': 1.520468869219889e-08}. Best is trial 11 with value: 0.6985100057396345.
[I 2023-07-03 03:14:50,615] Trial 47 finished with value: 0.657613082604906 and parameters: {'var_smoothing': 4.369946783595579e-07}. Best is trial 11 with value: 0.6985100057396345.
[I 2023-07-03 03:14:50,769] Trial 48 finished with value: 0.6590908499493066 and parameters: {'var_smoothing': 6.713854967599219e-07}. Best is trial 11 with value: 0.6985100057396345.
[I 2023-07-03 03:14:50,927] Trial 49 finished with value: 0.637636951348014 and p

[I 2023-07-03 03:14:57,173] Trial 89 finished with value: 0.657613082604906 and parameters: {'var_smoothing': 2.0207122587167362e-07}. Best is trial 69 with value: 0.7006162869650499.
[I 2023-07-03 03:14:57,328] Trial 90 finished with value: 0.6343826623866627 and parameters: {'var_smoothing': 6.873211713642716e-10}. Best is trial 69 with value: 0.7006162869650499.
[I 2023-07-03 03:14:57,485] Trial 91 finished with value: 0.6787926283480018 and parameters: {'var_smoothing': 9.833622008382918e-06}. Best is trial 69 with value: 0.7006162869650499.
[I 2023-07-03 03:14:57,640] Trial 92 finished with value: 0.6802997894996801 and parameters: {'var_smoothing': 2.1159009829538346e-05}. Best is trial 69 with value: 0.7006162869650499.
[I 2023-07-03 03:14:57,798] Trial 93 finished with value: 0.6604204754556747 and parameters: {'var_smoothing': 8.490639132761158e-07}. Best is trial 69 with value: 0.7006162869650499.
[I 2023-07-03 03:14:57,952] Trial 94 finished with value: 0.6818722497090264 an

In [251]:
# Get the best hyperparameters from Optuna
best_params = study.best_params

In [252]:
best_params #42 random seed

{'var_smoothing': 0.0008094845352286139}

#### Get best parameters and make predictions

In [253]:
#selected_features_indices = [i for i, value in enumerate(best_features) if value]
X_test_selected = X_test[:, selected_features]

In [254]:
X_test_selected.shape

(286, 300)

In [255]:
X_train_val_selected_final_features = X_train_val[:, selected_features]

In [256]:
X_train_val_selected_final_features.shape

(1141, 300)

In [257]:
#Define the best classifier

#best_C = study.best_params['C']
#best_gamma = study.best_params['gamma']
#best_gamma = study.best_params['gamma']
#best_classifier = SVC(C=best_C, gamma=best_gamma)
best_classifier_GNB = GaussianNB(**best_params)

In [269]:
#Fit the model and make predictions
best_classifier_GNB.fit(X_train_val_selected_final_features, y_train_val)

GaussianNB(var_smoothing=0.0008094845352286139)

In [259]:
y_test_pred = best_classifier_GNB.predict(X_test_selected)

In [260]:
#Validate the model  using balanced_acc, precision,recall and f1
balanced_acc_mrmr_GNB = balanced_accuracy_score(y_test, y_test_pred)
precision_mrmr_GNB = precision_score(y_test, y_test_pred,average='weighted')
recall_mrmr_GNB = recall_score(y_test, y_test_pred,average='weighted')
f1_mrmr_GNB = f1_score(y_test, y_test_pred,average='weighted')

In [261]:
balanced_acc_mrmr_GNB

0.8512058969063293

In [262]:
precision_mrmr_GNB

0.8667967659870494

In [263]:
recall_mrmr_GNB

0.8636363636363636

In [264]:
f1_mrmr_GNB

0.8615921762774909

In [265]:
y_test_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 3, 2, 3, 2, 2, 1, 1, 3, 3, 0, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 2, 3, 3, 2, 0, 2, 0, 3, 3, 2, 1, 3, 3, 3, 3,
       0, 2, 3, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 2, 2, 0, 2, 1, 3, 0, 0, 0, 3, 1,
       2, 2, 1, 0, 3, 2, 0, 1, 0, 2, 3, 2, 0, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 3, 3, 0, 0, 2, 2, 2, 0, 1, 1, 0, 3, 1, 2, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 1, 3, 3, 2, 2, 0, 3, 3, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 3, 2, 1, 1, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 2, 0, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 3, 2, 3, 1, 2, 3, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

In [266]:
y_test

array([0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 2, 3, 3, 0,
       0, 0, 3, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 1, 3, 2, 1, 3, 0, 3, 3,
       0, 2, 3, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2,
       2, 2, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1,
       2, 0, 1, 0, 3, 2, 0, 1, 0, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 2, 1, 0,
       0, 2, 2, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 3, 1, 1, 2, 2, 3, 3, 0, 0,
       3, 0, 0, 0, 0, 3, 0, 3, 3, 0, 2, 3, 3, 2, 2, 0, 3, 2, 2, 0, 0, 2,
       2, 1, 0, 3, 0, 2, 2, 1, 2, 0, 3, 1, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1,
       0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 3, 2, 1, 2, 3, 1, 0, 0,
       3, 2, 0, 3, 3, 1, 0, 2, 0, 0, 2, 0, 2, 0, 3, 1, 0, 3, 2, 3, 0, 3,
       0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1, 2, 3, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2])

# Plots


In [267]:
classifiers = ['LogisticReg', 'RandomForest', 'GaussiaNB', 'SVC', 'XGBoost']

## Balanced accuracy specific

In [ ]:
anova_scores = [balanced_acc_LR, balanced_acc_RF, balanced_acc_GNB, balanced_acc_SVC, balanced_acc_XGB]
mrmr_scores = [balanced_acc_mrmr_LR, balanced_acc_mrmr_RF, balanced_acc_mrmr_GNB, balanced_acc_mrmr_SVC, balanced_acc_mrmr_XGΒ]

In [ ]:
#Plot the Balanced accuracy for all the classifiers for the two methods of feature selection
#metric_index = metrics.index('balanced_acc')
plt.figure(figsize=(10, 6))
plt.plot(classifiers, anova_scores, marker='o', label='ANOVA')
plt.plot(classifiers, mrmr_scores, marker='o', label='mRMR')
plt.xlabel('Classifiers')
plt.ylabel('Balanced Accuracy')
plt.title('Comparison of Classifiers across Feature Selection Methods (Balanced Accuracy)')
plt.legend()
plt.show()

In [ ]:
anova_scores

In [ ]:
mrmr_scores

## Precision specific

In [ ]:
anova_scores_prec = [precision_LR, precision_RF, precision_GNB, precision_SVC, precision_XGB]
mrmr_scores_prec = [precision_mrmr_LR, precision_mrmr_RF, precision_mrmr_GNB, precision_mrmr_SVC, precision_mrmr_XGB]

In [ ]:
#Plot the Precision for all the classifiers for the two methods of feature selection
plt.figure(figsize=(10, 6))
plt.plot(classifiers, anova_scores_prec, marker='o', label='ANOVA')
plt.plot(classifiers, mrmr_scores_prec, marker='o', label='mRMR')
plt.xlabel('Classifiers')
plt.ylabel('Precision')
plt.title('Comparison of Classifiers across Feature Selection Methods (Precision)')
plt.legend()
plt.show()

In [ ]:
anova_scores_prec

In [ ]:
mrmr_scores_prec

## Recall

In [ ]:
anova_scores_rec = [recall_LR, recall_RF, recall_GNB, recall_SVC, recall_XGB]
mrmr_scores_rec = [recall_mrmr_LR, recall_mrmr_RF, recall_mrmr_GNB, recall_mrmr_SVC, recall_mrmr_XGB]

In [ ]:
#Plot the Recall for all the classifiers for the two methods of feature selection
plt.figure(figsize=(10, 6))
plt.plot(classifiers, anova_scores_rec, marker='o', label='ANOVA')
plt.plot(classifiers, mrmr_scores_rec, marker='o', label='mRMR')
plt.xlabel('Classifiers')
plt.ylabel('Recall')
plt.title('Comparison of Classifiers across Feature Selection Methods (Recall)')
plt.legend()
plt.show()

In [ ]:
anova_scores_rec

In [ ]:
mrmr_scores_rec

## F1 score

In [ ]:
anova_scores_f1 = [f1_LR, f1_RF, f1_GNB, f1_SVC, f1_XGΒ]
mrmr_scores_f1 = [f1_mrmr_LR, f1_mrmr_RF, f1_mrmr_GNB, f1_mrmr_SVC, f1_mrmr_XGΒ]

In [ ]:
#Plot the F1-score for all the classifiers for the two methods of feature selection
plt.figure(figsize=(10, 6))
plt.plot(classifiers, anova_scores_f1, marker='o', label='ANOVA')
plt.plot(classifiers, mrmr_scores_f1, marker='o', label='mRMR')
plt.xlabel('Classifiers')
plt.ylabel('F1-score')
plt.title('Comparison of Classifiers across Feature Selection Methods (F1-score)')
plt.legend()
plt.show()

In [ ]:
anova_scores_f1

In [ ]:
anova_scores_f1